In [2077]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones
import re

from unidecode import unidecode
import unicodedata
import datetime
import os
import gc


# visuzalizaciones
import missingno as msno
import matplotlib as plt
import plotly.express as px

In [2078]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows',80)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

# Funciones

In [2079]:


# Función para normalizar texto
def normalize_text(text):
    # Reemplazar caracteres especiales y convertir a minúsculas para una comparación consistente
    text = text.replace("/", " ")  # Reemplazar la barra por espacio o eliminarla completamente
    text = text.replace("ñ", "n")  # Reemplazar la "ñ" por "n"
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

In [2080]:
def normalizar(texto):
 return unidecode(texto).lower()

In [2081]:
def actualizar_valores(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.

    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()

    # Actualizar los valores en el dataframe principal
    df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])

    return df


In [2082]:
def actualizar_valores_conclave(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.

    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()

    # Actualizar los valores en el dataframe principal solo si coinciden las claves
    df[columna_a_actualizar] = df[clave].map(valores_correctos).combine_first(df[columna_a_actualizar])

    return df


In [2083]:
def obtener_valores_por_columna(df, columna, valor):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.

    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    # Convertir los valores de la columna a cadenas de texto y eliminar espacios en blanco
    df[columna] = df[columna].astype(str).str.strip()
    valor = str(valor).strip()

    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()
    else:
        return {}


In [2084]:
def porcentaje_nulos(df, columnas):
    if isinstance(columnas, str):
        columnas = [columnas]  # Convertir a lista si es una sola columna
    resultados = {}
    for columna in columnas:
        if columna in df.columns:
            total_filas = len(df)
            nulos = df[columna].isnull().sum()
            porcentaje_nulos = (nulos / total_filas) * 100
            resultados[columna] = porcentaje_nulos
        else:
            resultados[columna] = 'Column not found'
    return resultados



In [2085]:

# Función para clasificar el tipo de calefacción y asignar el nombre de la lista
def clasificar_calefaccion(calefaccion, listas):
    for tipo, lista in listas.items():
        if calefaccion in lista:
            return tipo
    return 'No clasif'

In [2086]:
def agrupar_cocinas(tipo):
    categorias = []
    if pd.isnull(tipo):
        categorias.append('sin_informacion')
    elif isinstance(tipo, str):
        if 'abierta' in tipo.lower() or 'abierto' in tipo.lower() or 'americana' in tipo.lower() or 'integrada' in tipo.lower():
            categorias.append('abierta')
        if 'independiente' in tipo.lower():
            categorias.append('independiente')
        if 'amplia' in tipo.lower():
            categorias.append('amplia')
        if 'isla' in tipo.lower():
            categorias.append('con isla')
        if 'amueblada' in tipo.lower():
            categorias.append('amueblada')
        if 'con electrodomesticos' in tipo.lower() or 'equipada' in tipo.lower() or 'con elec.' in tipo.lower():
            categorias.append('equipada')
        if 'sin amueblar' in tipo.lower():
            categorias.append('no amueblada')
        if 'reformada' in tipo.lower():
            return 'reformada'
        elif 'a reformar' in tipo.lower() or 'necesita reforma' in tipo.lower():
            return 'Sin reforma'
        if not categorias:
            categorias.append('sin categoria')
        return ', '.join(categorias)
    else:
        return 'error tipo campo no texto'
    return ', '.join(categorias)


In [2087]:

def clasificar_consumo(valor):
    try:
        valor = float(valor)
        if valor < 10:
            return 'A'
        elif valor <= 20:
            return 'B'
        elif valor <= 35:
            return 'C'
        elif valor <= 55:
            return 'D'
        elif valor <= 75:
            return 'E'
        elif valor <= 100:
            return 'F'
        else:
            return 'G'
    except ValueError:
        return None



In [2088]:
# Función para extraer solo el número de la cadena de texto
def extraer_numero(texto):
    if isinstance(texto, str):
        match = re.search(r'\d+', texto)
        return match.group(0) if match else None
    return None

# Carga de datos

In [2089]:

print(os.getcwd())

c:\Users\extas\Documents\GitHub\precios-de-vivienda-madrid\codigo\01-preprocesamiento-de-datos\Revision_columnas


In [2090]:
#df17= pd.read_csv('./data/dfpisosCompleto17012025.csv',index_col=False)
df17= pd.read_csv('./data/20250301_Union_todos.csv',sep=";",index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_7920\241202692.py:2: DtypeWarning: Columns (3,12,13,14,17,20,23,34,51,53,54,57,58,63,64,69,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df17= pd.read_csv('./data/20250301_Union_todos.csv',sep=";",index_col=False)


In [2091]:
df17["jardin"].value_counts(dropna=False)

jardin
NaN            8087
sin jardin     1575
comunitario     298
privado         245
Name: count, dtype: int64

In [2092]:
dfbarrios= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')

In [2093]:
dfadicionales= pd.read_csv('../02-datos-limpios/Datos_adicionales.csv',sep=";", index_col=False)

In [2094]:
# Reemplazar los valores 'na' por NaN
df17.replace('na', pd.NA, inplace=True)

In [2095]:
df17.columns

Index(['title', 'url', 'precio_anterior', 'descuento', 'EUR/m2', 'm2_constr',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona', 'calle',
       'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia', 'estado',
       'amueblado', 'planta', 'dormitorios', 'banos', 'balcon', 'terraza',
       'garaje', 'trastero', 'ascensor', 'calefaccion', 'antiguedad',
       'num_plantas', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'nuda_propiedad', 'tipo_inmueble',
       'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpinteria_exterior_vidrio/metal', 'chimenea', 'cocina',
       'fibra_optica', 'exterior', 'interi

In [2096]:
df17.shape

(10205, 89)

In [2097]:
df17.query("descripcion_larga.isnull() and web=='Indomio.com'").count()

title                42
url                  42
precio_anterior       0
descuento             0
EUR/m2               42
                     ..
letrace               0
portero               0
orientacion_este      0
orientacion_oeste     0
orientacion_sur       0
Length: 89, dtype: int64

In [2098]:
df17.query("descripcion_larga.isnull() and web=='pisosmadrid'").count()

title                15
url                  15
precio_anterior       0
descuento             0
EUR/m2                0
                     ..
letrace               0
portero               0
orientacion_este      0
orientacion_oeste     0
orientacion_sur       0
Length: 89, dtype: int64

In [2099]:
df17.query("descripcion_larga.isnull() and web=='redpiso'").count()

title                6
url                  6
precio_anterior      0
descuento            0
EUR/m2               0
                    ..
letrace              0
portero              0
orientacion_este     0
orientacion_oeste    0
orientacion_sur      0
Length: 89, dtype: int64

In [2100]:
df17.query("descripcion_larga.isnull() and web=='pisos.com'").count()

title                1863
url                  1863
precio_anterior         0
descuento               0
EUR/m2               1862
                     ... 
letrace                 0
portero                 0
orientacion_este        0
orientacion_oeste       0
orientacion_sur         0
Length: 89, dtype: int64

In [2101]:
# Chqueo de nulos en todas las columnas
df17.isna().sum()
# Muestreo estadistico de las columnas

title                    0
url                      0
precio_anterior      10150
descuento            10150
EUR/m2                2350
                     ...  
letrace              10203
portero              10006
orientacion_este     10204
orientacion_oeste    10204
orientacion_sur      10202
Length: 89, dtype: int64

In [2102]:
df17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10205 entries, 0 to 10204
Data columns (total 89 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   title                                      10205 non-null  object 
 1   url                                        10205 non-null  object 
 2   precio_anterior                            55 non-null     float64
 3   descuento                                  55 non-null     object 
 4   EUR/m2                                     7855 non-null   float64
 5   m2_constr                                  10198 non-null  float64
 6   distrito                                   10198 non-null  object 
 7   cod_distrito                               10198 non-null  float64
 8   barrio                                     10137 non-null  object 
 9   cod_barrio                                 10137 non-null  float64
 10  zona                  

In [2103]:
df17.describe().T

,count,mean,std,min,25%,50%,75%,max
precio_anterior,55.00,830694.55,977555.73,98100.00,201500.00,440000.00,955000.00,4900000.00
EUR/m2,7855.00,3745.76,3996.23,0.00,7.08,3020.00,6360.00,27708.00
m2_constr,10198.00,138.65,150.97,0.00,68.00,99.00,150.00,4574.00
cod_distrito,10198.00,7.78,5.77,1.00,4.00,6.00,12.00,21.00
cod_barrio,10137.00,80.95,57.73,1.00,36.00,64.00,123.00,215.00
dormitorios,9976.00,2.83,1.42,0.00,2.00,3.00,3.00,24.00
balcon,1461.00,0.58,0.49,0.00,0.00,1.00,1.00,1.00
terraza,2499.00,0.77,0.42,0.00,1.00,1.00,1.00,1.00
trastero,2724.00,0.50,0.50,0.00,0.00,1.00,1.00,1.00
ascensor,4638.00,0.91,0.29,0.00,1.00,1.00,1.00,1.00


In [2104]:
df17["url"].isna().sum()

0


# Filtro de las filas con valores en precio anteior
filas_sin_nulos= df17[~df17["precio_anterior"].isnull()]
filas_sin_nulos.head(4)
# Aqui filtre los links de los inmuebles con el precio anterior para checar algunos directamente y
# ver si habia algo relevante del porque del cambio del precio.
filas_sin_nulos["url"]
# Chequeo de porcentaje de nulos en la columna precio_anterior
df17["precio_anterior"].isna().mean()*100
df17.shape
# Porcentaje de nulos
df17["descuento"].isna().mean()*100
#Porcentaje de nulos
df17["EUR/m2"].isna().mean()*100
# Chequeo de nulos totales
df17["EUR/m2"].isna().sum()
df17["EUR/m2"].describe()
# Filtro de valores iguales a 0
filas_valor_eur= df17[df17["EUR/m2"] == 0]
filas_valor_eur

In [2105]:
# Filtro de valores
#filas_valor_eur_pequeños= df_131[df_131["EUR/m2"] <=100 ]
#filas_valor_eur_pequeños.tail(5)
# Filtro para valores menores a 99 con un decimal. de esa forma los valores podran ser filtrados para no tener un precio mayor a 4 cifras por metro cuadrado
# que es lo que se ha notado donde predominan los valores.  Aqui cheque los valores con Data Wrangler y pude visualizar que los valores no son correctos. A algunos
# les sirve solo colocarle 000 ceros extras al valor por metro cuadrado pero a otros no les sirve eso.

#filtered_df_131 = df_131[df_131['EUR/m2'].apply(lambda x: x < 99 and (x * 10).is_integer())]
#filtered_df_131.head(5)
# Filtro por valores que parecen atipicos e irregulares para identificar vericidad
#filas_valor_eur2= df_131[df_131["EUR/m2"] >= 20000]
#filas_valor_eur2.head()
# Filtro del link por valores que parecen atipicos e irregulares para identificar vericidad
#filas_valor_eur2= df_131[df_131["EUR/m2"] >= 20000]
#filas_valor_eur2["url"]
# Porcentaje de nulos por columna
#df_131.isna().mean()*100

### cod_barrio

In [2106]:
df17.loc[df17['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3863841-piso-en-venta-en-madrid'), ['cod_distrito','distrito','cod_barrio','barrio']] =[13,'Puente de Vallecas',133,'Palomeras Bajas']
df17.loc[df17['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4086493-piso-en-venta-en-madrid'), ['cod_distrito','distrito','cod_barrio','barrio']] =[17,'Villaverde',171,'Villaverde Alto']

## m2_constr

In [2107]:
# Analisis descriptivo de la columna m2/constr
df17["m2_constr"].describe()

count   10198.00
mean      138.65
std       150.97
min         0.00
25%        68.00
50%        99.00
75%       150.00
max      4574.00
Name: m2_constr, dtype: float64

In [2108]:

#Filtro de valores menores a 20 metros cuadrados. Se analizo con 10 tambien y siguen siendo mas de 200
df17[df17["m2_constr"] <= 10].count()

title                3
url                  3
precio_anterior      0
descuento            0
EUR/m2               1
                    ..
letrace              0
portero              0
orientacion_este     0
orientacion_oeste    0
orientacion_sur      0
Length: 89, dtype: int64

#en la web ponen que tiene 10 m2, y por las fotos lo parece

In [2109]:
#voy a mirar los valores de m2_constr>1000
df17.query("m2_constr>= 4000")[['url','dormitorios','m2_constr']]

,url,dormitorios,m2_constr
2394,https://www.indomio.es/anuncios/96765659/,10.00,4574.00


In [2110]:
# Eliminar la fila donde la columna 'url' es igual a la URL especificada
df17 = df17[df17["url"] != "https://www.indomio.es/anuncios/96246263/"]


In [2111]:
# Resetear el índice del DataFrame
df17.reset_index(drop=True, inplace=True)

In [2112]:
#en la informacion dela web https://www.pisos.com/comprar/chalet-moncloa_aravaca_aravaca28023-4710772760_109700/ se dice 800 m2

# Actualizar el valor de m2_contr para la fila cuya URL contiene la cadena proporcionada
url_especifica = 'https://www.pisos.com/comprar/chalet-moncloa_aravaca_aravaca28023-'
#dfEDA.loc[dfEDA['url'].str.contains(url_especifica), ['m2_constr','dormitorios','banos','tiene_piscina',"num_garajes","trastero","estado"] = [800,5,5,1,2,1,'Obra nueva']
df17.loc[df17['url'].str.contains(url_especifica), ['m2_constr', 'dormitorios', 'banos', 'tiene_piscina', 'num_garajes', 'trastero', 'estado']] = [800, 5, 5, 1, 2, 1,'Obra nueva']
url_especifica='https://www.indomio.es/anuncios/95261315/'
df17.loc[df17['url'].str.contains(url_especifica), ['m2_constr',  'tiene_piscina', 'num_garajes', 'trastero', 'estado']] = [271,1,0, 0,'Buen Estado']
url_especifica='https://www.indomio.es/anuncios/94926741/'
df17.loc[df17['url'].str.contains(url_especifica), ['nuda_propiedad',  'tiene_piscina', 'num_garajes', 'trastero']] = [1,0,0,0]
#Elimino https://www.indomio.es/anuncios/91050501/ por que es un edificio con 5 estudios, no tenemos metros
url_a_eliminar = 'https://www.indomio.es/anuncios/91050501/'
df17 = df17[df17['url'] != url_a_eliminar]
df17.loc[df17["m2_constr"] == "3 baños", "m2_constr"] = np.nan

# se elimina la url https://www.pisos.com/comprar/piso-fontarron-47524209698_100600/ por que es un restaurante en venta



In [2113]:
df17.reset_index(drop=True, inplace=True)

In [2114]:
#quedan 5 viviendas sin m2_constr

#### distrito

In [2115]:
df17['distrito'].value_counts(dropna=False)

distrito
Centro                 1808
Salamanca              1532
Chamartín               640
Chamberí                612
Carabanchel             541
Puente De Vallecas      529
Tetuán                  467
Fuencarral-El Pardo     432
Retiro                  428
Latina                  416
Ciudad Lineal           409
Moncloa-Aravaca         380
San Blas-Canillejas     350
Arganzuela              298
Hortaleza               291
Villaverde              291
Villa De Vallecas       233
Usera                   196
Vicálvaro               137
Moratalaz               105
Barajas                 100
NaN                       7
Puente de Vallecas        1
Name: count, dtype: int64

In [2116]:
df17["distrito"].unique()

array(['Salamanca', 'Barajas', 'Arganzuela', 'Chamartín', 'Tetuán',
       'Chamberí', 'Villaverde', 'Latina', 'Centro',
       'Fuencarral-El Pardo', 'Villa De Vallecas', 'Puente De Vallecas',
       'Retiro', 'San Blas-Canillejas', 'Ciudad Lineal', 'Hortaleza',
       'Moncloa-Aravaca', 'Carabanchel', 'Vicálvaro', 'Usera',
       'Moratalaz', 'Puente de Vallecas', nan], dtype=object)

In [2117]:
# Ajustar la configuración para mostrar más caracteres en las columnas
pd.set_option('display.max_colwidth', None)

### nuda propiedad son imuebles sin posesion, se estan incluyendo inmuebles que se tiene la posesion pero no el uso y disfrute por que alguien tiene el usufructo, tabien se incluye inmuebles ocupados

In [2118]:
df17.query("cod_distrito.isnull()")[['distrito','url','cod_distrito','barrio','cod_barrio','m2_constr','planta','dormitorios','banos','precio','estado','tipologia','terraza']]

,distrito,url,cod_distrito,barrio,cod_barrio,m2_constr,planta,dormitorios,banos,precio,estado,tipologia,terraza
9391,NaN,https://www.pisosmadrid.com.es/propiedad/3890778-piso-en-venta-en-madrid,NaN,NaN,NaN,26.00,NaN,0.00,0.00,79900,NaN,Piso,NaN
9584,NaN,https://www.pisosmadrid.com.es/propiedad/3961341-piso-en-venta-en-madrid,NaN,NaN,NaN,109.00,NaN,1.00,1.00,138600,NaN,Piso,NaN
9592,NaN,https://www.pisosmadrid.com.es/propiedad/3963177-piso-en-venta-en-arturo-soria-madrid,NaN,NaN,NaN,70.00,NaN,1.00,1.00,450410,NaN,Piso,NaN
9680,NaN,https://www.pisosmadrid.com.es/propiedad/3998979-piso-en-venta-en-vallecas-madrid,NaN,NaN,NaN,98.00,NaN,2.00,2.00,205000,NaN,Piso,NaN
9794,NaN,https://www.pisosmadrid.com.es/propiedad/4029531-piso-en-venta-en-madrid,NaN,NaN,NaN,81.00,NaN,3.00,1.00,295000,NaN,Piso,NaN
9839,NaN,https://www.pisosmadrid.com.es/propiedad/4042093-piso-en-venta-en-madrid,NaN,NaN,NaN,139.00,NaN,3.00,2.00,426000,NaN,Piso,NaN
9960,NaN,https://www.pisosmadrid.com.es/propiedad/4056566-piso-en-venta-en-madrid,NaN,NaN,NaN,73.00,NaN,2.00,2.00,395000,NaN,Piso,NaN


No tenemos forma de conocer la ubicacion de https://www.pisosmadrid.com.es/propiedad/3930386-piso-en-venta-en-madrid	, lo elimino del dataset

In [2119]:
df17.query("cod_barrio.isnull() & cod_distrito.notnull()")[['distrito','cod_distrito','barrio','cod_barrio','descripcion_larga','url']]

,distrito,cod_distrito,barrio,cod_barrio,descripcion_larga,url
3894,Vicálvaro,19.00,NaN,NaN,"VI-1-2-1-P01-A. Piso en un edificio de obra nueva de 110 m2 de superficie construida y ubicado en la zona de Madrid Capital, enclavado en planta 1. Está distribuido en 3 habitaciones, 2 baños completos y terraza de 7,3 m2. Incluye extras como ascensor, garaje y trastero.** El precio mostrado en la tabla anterior para cada inmueble, incluye únicamente aquellos anejos (plazas de garaje y/o trastero) que hubiesen sido expresamente especificados en su descripción. En caso de no aparecer especificados, su precio no se entenderá incluido en el precio mostrado anteriormente. El precio mostrado lo es sin incluir impuestos. Los inmuebles identificados están sujetos, en todo caso, a disponibilidad.",https://www.pisos.com/comprar/piso-los_berrocales28052-4592114246_109700/
4060,Vicálvaro,19.00,NaN,NaN,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4596004879_109700/
4061,Vicálvaro,19.00,NaN,NaN,"P2_1ºC. Vivienda de 2 dormitorios en Planta Primera, 77m² construidos, 57,50m² útiles y terraza 6,80m². Con precio desde 301.500€ más (10% IVA), 1 plaza de garaje y 1 trastero incluido en el precio.",https://www.pisos.com/comprar/piso-los_berrocales28031-4849665223_109700/
4062,Vicálvaro,19.00,NaN,NaN,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4510709962_109700/
4063,Vicálvaro,19.00,NaN,NaN,"P2_1ºB. Vivienda de 3 dormitorios en Planta Primera, 99m² construidos, 75,40m² útiles y terraza 4,5m². Con precio de 371.900€ más (10% IVA), 2 plazas de garaje y 1 trastero incluido en el precio.",https://www.pisos.com/comprar/piso-los_berrocales28031-4891530918_109700/
4192,Vicálvaro,19.00,NaN,NaN,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-3500496548_109700/
4193,Vicálvaro,19.00,NaN,NaN,NaN,https://www.pisos.com/comprar/piso-los_berrocales28031-4568166771_109700/
7612,Vicálvaro,19.00,NaN,NaN,"Portal 1, 1 A. Piso en una finca de obra nueva de 90 m2 de superficie construida y emplazado en la zona de Madrid Capital, posicionado en planta 1. Está dispuesto en 2 habitaciones, 2 baños completos y terraza. Cuenta como extras garaje y trastero.",https://www.pisos.com/comprar/piso-los_berrocales28031-4677433362_109700/
7613,Vicálvaro,19.00,NaN,NaN,"Portal 1, 1 B. Piso en una finca de obra nueva de 92 m2 de superficie construida y emplazado en la zona de Madrid Capital, posicionado en planta 1. Está dispuesto en 2 habitaciones, 2 baños completos y terraza. Cuenta como extras garaje y trastero.",https://www.pisos.com/comprar/piso-los_berrocales28031-4588865732_109700/
7614,Vicálvaro,19.00,NaN,NaN,"Vivienda de 2 dormitorios en Planta Primera, 85m² construidos, 56,20m² útiles y terraza 7,30m². Con precio de 320.200€ (más 10% IVA), 1 plaza de garaje y 1 trastero, incluido en el precio.",https://www.pisos.com/comprar/piso-los_berrocales28031-4355464537_109700/


In [2120]:
#creo una columna en el dataframe dfbarrios para normalizar los distritos
# Eliminar acentos de la columna 'distrito'
#dfbarrios['distrito_n'] = dfbarrios['distrito'].apply(lambda x: unidecode(x))
#dfbarrios['barrio_n'] = dfbarrios['barrio'].apply(lambda x: unidecode(x))
#dfbarrios['barrio_nn']=dfbarrios['barrio'].apply(lambda x: normalizar(x))


In [2121]:
#dfbarrios.query("distrito==' Vicálvaro'")

# Reemplazar la palabra "De" por "de" en la columna 'distrito'
df17['distrito'] = df17['distrito'].str.replace(r'\bDe\b', 'de', regex=True)


# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("(cod_barrio.isnull() & cod_distrito.isnull()) | (cod_barrio.isnull() & cod_distrito == 0)")
#ahora tengo nombre distrito pero no codigo
# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():

    bar= unidecode(fila['distrito'])


    valores = obtener_valores_por_columna(dfbarrios, 'distrito_n', bar)

    if valores:
        #if pd.isna(df17.at[idx, 'barrio']):
         #   df17.at[idx, 'barrio'] = valores.get('barrio')
        #if pd.isna(df17.at[idx, 'cod_barrio']):
         #   df17.at[idx, 'cod_barrio'] = valores.get('cod_barrio')
        if pd.isna(df17.at[idx, 'cod_distrito']):
            df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
        if pd.isna(df17.at[idx, 'distrito']):
            df17.at[idx, 'distrito'] = valores.get('distrito')
        #df17.at[idx, 'zona'] = 'bar'


filas = df17.query("cod_barrio.isnull() ")

# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
    if pd.notna(fila['zona']):
        bar = unidecode(fila['zona'])
        if bar=='villaverdealto,cascohistoricodevillaverde':
            bar='villaverde alto'
        if bar=='loscarmenes':
            bar='los carmenes'
        if bar=='angeles':
            bar='los angeles'
        if bar=='cascohistoricodevicalvaro':
            bar='casco historico de vicalvaro'
        if bar=='elsalvador':
            bar='salvador'
        if bar=='Malasana':
            bar='universidad'
        if bar=='Ventilla':
            bar='almenara'
        if bar=='Orense':
            bar='castillejos'
        if bar=='VillaverdeBajo':
            bar='los rosales'
        if bar=='LaLatina':
            bar='palacio'

       #print (fila['zona'])
        valores = obtener_valores_por_columna(dfbarrios, 'barrio_nn', bar)
        print(bar)
        if valores:
           # if bar=='valdefuentes':
            print(bar)
            #if pd.isna(df17.at[idx, 'cod_distrito']):
            n= valores.get('cod_distrito')
            df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
            #if pd.isna(df17.at[idx, 'distrito']):
            df17.at[idx, 'distrito'] = valores.get('distrito')
            #if pd.isna(df17.at[idx, 'barrio']):
            df17.at[idx, 'barrio'] = valores.get('barrio')
            #if pd.isna(df17.at[idx, 'cod_barrio']):
            df17.at[idx, 'cod_barrio'] = valores.get('cod_barrio')
            df17.at[idx, 'codigo_Postal'] = valores.get('codigo_Postal')

# Filtrar filas donde 'cod_barrio' no es nulo y 'cod_distrito' es nulo
filas = df17.query("cod_barrio.isnull() ")

# Iterar sobre las filas filtradas
for idx, fila in filas.iterrows():
    if pd.notna(fila['zona']):
        bar = unidecode(fila['zona'])
        if bar=='villaverdealto,cascohistoricodevillaverde':
            bar='villaverde alto'
        if bar=='loscarmenes':
            bar='los carmenes'
        if bar=='angeles':
            bar='los angeles'
        if bar=='cascohistoricodevicalvaro':
            bar='casco historico de vicalvaro'
        if bar=='elsalvador':
            bar='salvador'
        if bar=='Malasana':
            bar='universidad'
        if bar=='Ventilla':
            bar='almenara'
      #print (fila['zona'])
        valores = obtener_valores_por_columna(dfbarrios, 'distrito_n', bar)
        print(bar)
        if valores:
           # if bar=='valdefuentes':
            print(bar)
            #if pd.isna(df17.at[idx, 'cod_distrito']):
            n= valores.get('cod_distrito')
            df17.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
            #if pd.isna(df17.at[idx, 'distrito']):
            df17.at[idx, 'distrito'] = valores.get('distrito')


In [2122]:
df17.query("cod_barrio.isnull()").count()

title                68
url                  68
precio_anterior       0
descuento             0
EUR/m2               19
                     ..
portero               0
orientacion_este      0
orientacion_oeste     0
orientacion_sur       0
num_garajes           0
Length: 90, dtype: int64

In [2123]:
#como en el origen está bien, traigo los valores de pisos madrid
#dfInd1limpio= pd.read_csv('../02-datos-limpios/dUnionSilvia23122024.csv',index_col=False)

In [2124]:
#dfInd1limpio.query("cod_barrio.isnull()")[['distrito','cod_distrito','barrio','cod_barrio','read_all']]

In [2125]:
df17['num_visitas'].value_counts(dropna=False)

num_visitas
NaN        7885
3.00        149
4.00        139
2.00        126
5.00         94
           ... 
2175.00       1
414.00        1
2279.00       1
322.00        1
1923.00       1
Name: count, Length: 1103, dtype: int64

In [2126]:
df17["portero"].value_counts(dropna=False)

portero
NaN                      10004
Portero todo el día        145
1.0                         41
1                            6
Portero media jornada        4
todo el dia                  2
media jornada                1
Name: count, dtype: int64

In [2127]:

df17['portero'] = df17['portero'].replace('1', 'Portero todo el día')
df17['portero'] = df17['portero'].replace('1.0', 'Portero todo el día')
df17['portero'] = df17['portero'].replace('todo el dia', 'Portero todo el día')
df17['portero'] = df17['portero'].replace('media jornada', 'Portero media jornada')

## gastos comunidad

In [2128]:

df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('Ningún gasto de comunidad', '0')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace(',00', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('.', '').str.strip()
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('37,50', '38').str.strip()

##  terraza

In [2129]:
df17["terraza"].value_counts(dropna=False)

terraza
NaN     7705
1.00    1929
0.00     569
Name: count, dtype: int64

In [2130]:
df17['tiene_armario'] = df17['tiene_armario'].replace(0.0, np.nan)
df17['tiene_piscina'] = df17['tiene_piscina'].replace(0, np.nan)

# calefaccion

In [2131]:
df17["calefaccion"].value_counts(dropna=False)

calefaccion
NaN                                                   6724
Central                                                905
Individual                                             583
1.0                                                    565
Gas natural                                            520
 Gas Natural                                           251
 Eléctrica                                             123
Eléctrica                                              113
 Individual                                            104
 Central                                                87
Individual, alimentación eléctrica                      34
Individual, por radiadores, de gas                      30
Individual - gas natural                                18
Central, por radiadores, de gas                         13
Suelo radiante                                          11
Individual, por radiadores                              10
Gasoil                                      

In [2132]:


# Supongamos que df17 es tu DataFrame
#df17['portero'] = df17['portero'].replace('0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('0.0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('Climatización', 'climatizacion')
df17['calefaccion'] = df17['calefaccion'].replace('Climatizaciòn', 'climatizacion')
df17['calefaccion']=df17['calefaccion'].str.strip()

In [2133]:


# Palabras clave para clasificar
central_keywords = ["central", "comunitaria", "comunitario"]
individual_keywords = ["individual", "particular", "chimenea", "bomba", "termo", "climatizacion"]
electric_keywords = ["electrica", "bomba", "climatización", 'termo electrico','climatizacion']
gas_keywords = ["gas", "propano", "g n", "g/n","G/n"]  # Asegurarse de que 'g n' y 'g/n' estén bien escritos
gasoil_keywords = ["gasóil","gasoil",'gasoil',"Gasoil"]  # Añadir "gasoil" aquí
renovables_keywords = ["aerotermia", "geotermia", "chimenea"]
suelo_radiante_keywords=["suelo_radiante", "suelo"]
#radiadores_keywords=["radiadores"]
aire_keywords=["bomba","climatización",'climatizacion','conducto']
desconocido_keywords=['chimenea', 'termo electrico']
#lo mas extendido son los radiadores por eso despues de pasar , suelo radiante y aire pongo a todos radiadores salvo a chimenea y termo electrico que no lo se
radiadores_keywords=['radiadores','Gas natural', 'G/n', 'Gas particular', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Comunitario', 'Individual con gas natural', 'Individual, de gasóil', 'Comunitaria', 'Calefaccion Propano', 'Calefaccion Gas Natural', 'Central, por aire, de gas', 'Gas natural y biomasa', 'Centralizada', 'Eléctrica', 'Calefaccion Individual', 'Calefaccion individual','Contadores individuales', '1.0', 'Individual, alimentación eléctrica', 'Aerotermia', 'Individual - gas natural', 'Individual', 'Calefaccion Aerotermia', 'Gasoil', 'Central individualizada', 'Individual, por aire, alimentación eléctrica', 'Geotermia y aerotermia', 'Calefaccion comunitaria', 'Calefaccion Central', 'Central, de gas', 'Calefaccion Electrica', 'Calefacción central individualizada', 'Central, de gasóil', 'Individual, de gas', 'Central']

# Inicializar conjuntos para valores únicos
central = set()
individual = set()
electrica = set()
gas = set()
gsoil = set()
renovables = set()
suelo_radiante = set()
radiadores = set()
aire = set()
sin_clasificar = set()
sin_clasificar_energia = set()
sin_clasificar_medio = set()
radiadores=set()
suelo_radiante=set()


# Clasificar categorías
for categoria in df17["calefaccion"]:
    categoria_normalized = normalize_text(str(categoria))  # Normalizar texto

    # Clasificación por tipo (central o individual)
    if any(keyword in categoria_normalized for keyword in central_keywords):
        central.add(categoria)
    elif any(keyword in categoria_normalized for keyword in individual_keywords):
        individual.add(categoria)
    else:
        sin_clasificar.add(categoria)

    # Clasificación por energía (eléctrica, gas, gasoil)
    if any(keyword in categoria_normalized for keyword in electric_keywords):
        electrica.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gasoil_keywords):
        gsoil.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gas_keywords):
        gas.add(categoria)


    elif any(keyword in categoria_normalized for keyword in renovables_keywords):
        renovables.add(categoria)
    else:
        sin_clasificar_energia.add(categoria)

    # Clasificación por medio (suelo radiante, radiadores, aire)


    if any(keyword in categoria_normalized for keyword in aire_keywords):
        aire.add(categoria)
    elif any(keyword in categoria_normalized for keyword in suelo_radiante_keywords):
        suelo_radiante.add(categoria)
    elif any(keyword in categoria_normalized for keyword in desconocido_keywords):
        sin_clasificar_medio.add(categoria)
  #  else: any(keyword in categoria_normalized for keyword in radiadores_keywords):
    else:
        radiadores.add(categoria)


# Convertir los conjuntos a listas
central = list(central)
individual = list(individual)
sin_clasificar = list(sin_clasificar)
electrica = list(electrica)
gas = list(gas)
gasoil = list(gsoil)
renovables = list(renovables)
sin_clasificar_energia = list(sin_clasificar_energia)
aire=list(aire)
radiadores=list(radiadores)
suelo_radiante=list(suelo_radiante)
sin_clasificar_medio=list(sin_clasificar_medio)
# Imprimir resultados
print("Central:", central)
print("Individual:", individual)
print("No clasificado:", sin_clasificar)
print("Electrica:", electrica)
print("Gas:", gas)
print("Gasoil:", gasoil)
print("Renovables:", renovables)
print("No clasificado en energía:", sin_clasificar_energia)
print("Suelo Radiante:", suelo_radiante)
print("Radiadores:", radiadores)
print("Aire:", aire)
print("No clasificado en medio:", sin_clasificar_medio)


Central: ['Calefaccion comunitaria', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Calefacción central individualizada', 'Central, por radiadores', 'Central individualizada', 'Central, de gas', 'Central, de gasóil', 'Central, por radiadores, de gas', 'Central, por radiadores, de gasóil', 'Centralizada', 'central', 'Comunitaria', 'Central, suelo radiante', 'Central, por aire, de gas', 'central por gas', 'Central', 'Central, suelo radiante, de gas', 'Comunitario']
Individual: ['Individual - gas natural', 'Bomba frio/calor', 'Chimenea', 'Gas particular', 'Individual, por radiadores, de gasóil', 'Por climatizacion', 'Individual, alimentación eléctrica', 'Individual, por radiadores', 'Individual, suelo radiante, de gas', 'Bomba frio-calor', 'Individual, de gas', 'Individual, por radiadores, de gas', 'Individual, por aire, por bomba de calor', 'Contadores individuales', 'Climatizacion', 'Individual, suelo radiante', 'Individual, de gasóil', 'indivi

In [2134]:
df17["antiguedad"].value_counts(dropna=False)

antiguedad
NaN                    6762
Más de 50 años         1261
Entre 30 y 50 años      208
1960                    109
1900                    109
                       ... 
1916                      1
Entre 10 y  15 Años       1
 1970                     1
 1979                     1
2007.0                    1
Name: count, Length: 165, dtype: int64

In [2135]:
df17["num_plantas"].value_counts(dropna=False)

num_plantas
NaN           9970
6.0             40
4.0             39
3.0             28
7.0             24
5.0             23
8.0             19
10.0            13
2.0             13
Plantas: 5       6
9.0              6
Plantas: 4       5
1.0              5
Plantas: 6       3
Plantas: 3       2
12.0             1
11.0             1
23.0             1
Plantas: 2       1
Plantas: 7       1
Plantas: 1       1
Plantas: 8       1
Name: count, dtype: int64

In [2136]:
df17["num_plantas"]=df17["num_plantas"].str.replace("Plantas: ","",regex=False)

## aire acondicionado

In [2137]:
df17["aire_acondicionado"].value_counts(dropna=False)

aire_acondicionado
NaN                                                   7642
Individual                                             583
Frío                                                   480
1.0                                                    465
Individual, frío/calor                                 399
Frío y calor                                           320
Individual, frío                                       115
Splites repartidos                                      70
Independiente                                           26
Frío/calor                                              20
Centralizada                                            16
Suelo refrigerante                                      10
Frío-calor                                               7
Preinstalación                                           6
Preinstalado                                             5
Aerotermia                                               5
Central                              

In [2138]:
df17["aire_acondicionado"].unique()


array([nan, 'Preinstalación', 'Individual, frío/calor',
       'Individual, frío', 'Individual', 'Frío/calor',
       'Centralizada, frío', 'Centralizada, frío/calor',
       'Centralizada, calor', 'Centralizada', 'Frío', 'Frío y calor',
       'Splites repartidos', 'Aerotermia', 'A/a f/c',
       'Con bomba de calor', 'Preinstalado', 'Independiente',
       'Suelo refrigerante', 'Frío-calor', 'Sistema air zone',
       'Geotermia y aerotermia',
       'Climatizacion por conductos frio y calor',
       'Tiene la instalción hecha en toda la casa por cond',
       'Suelo radiante refrescante', 'No disponible', 'Central',
       'Frió/calor por conductos', 'Suelo radiante',
       'En toda la casa, por conductos',
       'Aire frio/ calor por conductos/suelo refrescante',
       'Por conductos en toda la casa', '1.0', 'por splits', '1',
       'bomba frio calor'], dtype=object)

In [2139]:
# Listas de valores para cada grupo
eficientes_val_columna = [
    'Aerotermia', 'Geotermia y aerotermia', 'Suelo radiante refrescante', 'Suelo refrigerante',
    'Sistema air zone', 'Climatizacion por conductos frio y calor', 'Frió/calor por conductos',
    'Suelo radiante', 'Aire frio/ calor por conductos/suelo refrescante', 'Por conductos en toda la casa',
    'Tiene la instalción hecha en toda la casa por cond', 'En toda la casa, por conductos'
]

normales_val_columna = [
    'Individual, frío/calor', 'Individual, frío', 'Individual', 'Frío/calor', 'Centralizada, frío',
    'Centralizada, frío/calor', 'Centralizada, calor', 'Central', 'Frío', 'Frío y calor',
    'Splites repartidos', 'Independiente', 'Con bomba de calor', 'A/a f/c',
    'Aire frio/calor por conductos/suelo refrescante', '1', '1.0'
]

sin_aire_val_columna = ['Preinstalación', 'Preinstalado', 'No disponible', '0', '0.0']

# Función para clasificar los sistemas
def clasificar_sistema(valor):
    if valor in eficientes_val_columna:
        return 'Eficiente'
    elif valor in normales_val_columna:
        return 'Normal'
    elif valor in sin_aire_val_columna:
        return 'Sin Aire'
    else:
        return 'Desconocido'  # Si el valor no coincide con ninguno de los grupos




In [2140]:
# Crear una nueva columna en el DataFrame
df17['clasificacion_aire'] = df17['aire_acondicionado'].apply(clasificar_sistema)


In [2141]:
df17["jardin"].unique()

array([nan, 'privado', 'sin jardin', 'comunitario'], dtype=object)

In [2142]:
# Ejemplo de uso con una sola columna
porcentaje = porcentaje_nulos(df17, 'num_plantas')
print(f"Porcentaje de valores nulos en 'num_plantas': {porcentaje.get('num_plantas', 'Column not found'):.2f}%")


Porcentaje de valores nulos en 'num_plantas': 97.72%


In [2143]:

# Ejemplo de uso
columnas = ["inmueble_ingresos", "alquiler_opcion_a_compra", "nuda_propiedad"]  # Reemplaza con las columnas de tu DataFrame
porcentaje_resultados = porcentaje_nulos(df17, columnas)

# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en inmueble_ingresos: 99.90%
Porcentaje de nulos en alquiler_opcion_a_compra: 99.97%
Porcentaje de nulos en nuda_propiedad: 99.87%


La columna disponibilidad no aporta nada

In [2144]:
df17["alquiler_opcion_a_compra"].value_counts(dropna=False)

alquiler_opcion_a_compra
NaN     10200
1.00        3
Name: count, dtype: int64

In [2145]:
df17.query("(inmueble_ingresos == 1) | (alquiler_opcion_a_compra == 1) | (nuda_propiedad == 1)").count()


title                 24
url                   24
precio_anterior        0
descuento              0
EUR/m2                12
                      ..
orientacion_este       0
orientacion_oeste      0
orientacion_sur        0
num_garajes            1
clasificacion_aire    24
Length: 91, dtype: int64

In [2146]:
df17['tipo_inmueble'].value_counts(dropna=False)

tipo_inmueble
NaN          9638
económico     276
medio         172
señorial      117
Name: count, dtype: int64

In [2147]:
df17["lujosa"].value_counts(dropna=False)

lujosa
NaN     7060
0.00    1715
1.00    1428
Name: count, dtype: int64

In [2148]:


# Aplicar la lógica: si 'lujosa' es 1 y 'tipo_inmueble' es nulo (NaN), entonces 'tipo_inmueble' = 'lujo'
df17['tipo_inmueble'] = df17.apply(
    lambda fila: 'lujo' if fila['lujosa'] == 1.00 and pd.isna(fila['tipo_inmueble']) else fila['tipo_inmueble'], axis=1
)



In [2149]:
df17=df17.drop(columns='lujosa')

In [2150]:
df17.columns

Index(['title', 'url', 'precio_anterior', 'descuento', 'EUR/m2', 'm2_constr',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona', 'calle',
       'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia', 'estado',
       'amueblado', 'planta', 'dormitorios', 'banos', 'balcon', 'terraza',
       'garaje', 'trastero', 'ascensor', 'calefaccion', 'antiguedad',
       'num_plantas', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'nuda_propiedad', 'tipo_inmueble',
       'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpinteria_exterior_vidrio/metal', 'chimenea', 'cocina',
       'fibra_optica', 'exterior', 'interi

In [2151]:
df17['tiene_armario'].value_counts(dropna=False)

tiene_armario
NaN     8401
1.00    1802
Name: count, dtype: int64

In [2152]:
df17['armarios'].value_counts(dropna=False)

armarios
 NaN                      8477
 1                         810
 Más de 2                  254
2.00                       143
1.00                       120
3.00                       104
 2                          71
4.00                        70
 1.0                        42
 2.0                        24
5.00                        23
6.00                        18
 3.0                        17
 5.0                         6
 4.0                         6
7.00                         5
8.00                         3
 8.0                         3
 6.0                         1
 3 empotrados                1
 8 armarios empotrados       1
 4 empotrados                1
 Mas de 5                    1
 1 closet                    1
9.00                         1
Name: count, dtype: int64

In [2153]:

# Lista de palabras a reemplazar
palabras_a_eliminar = ["empotrados", "armarios", "closet"]

# Crear una expresión regular que abarque todas las palabras
patron = '|'.join(palabras_a_eliminar)

# Reemplazar las palabras por vacío
df17['armarios'] = df17['armarios'].str.replace(patron, '', regex=True)




In [2154]:
# Aplicar la lógica: si 'armarios' no es NaN, entonces 'tiene_armario' = 1
df17['tiene_armario'] = df17.apply(
    lambda fila: 1 if pd.notna(fila['armarios']) and fila['armarios'] != '' else fila['tiene_armario'], axis=1
)

In [2155]:
df17['cancha_tenis'].value_counts(dropna=False)

cancha_tenis
NaN     7061
0.00    3140
1.00       2
Name: count, dtype: int64

In [2156]:
df17['chimenea'].value_counts(dropna=False)

chimenea
NaN     7057
0.00    3097
1.00      49
Name: count, dtype: int64

In [2157]:
df17[df17['chimenea'] == 1][['calefaccion', 'chimenea']]

,calefaccion,chimenea
22,NaN,1.00
80,NaN,1.00
207,"Central, por radiadores, de gas",1.00
211,"Individual, por radiadores, de gas",1.00
226,NaN,1.00
334,NaN,1.00
385,NaN,1.00
440,"Central, por radiadores, de gas",1.00
514,NaN,1.00
515,NaN,1.00


In [2158]:
df17['fibra_optica'].value_counts(dropna=False)

fibra_optica
NaN     7061
0.00    3104
1.00      38
Name: count, dtype: int64

In [2159]:
df17['carpinteria_exterior_doble_vidrio/pvc'].value_counts(dropna=False)

carpinteria_exterior_doble_vidrio/pvc
NaN     6703
0.00    3126
1.00     374
Name: count, dtype: int64

## cocina

In [2160]:
df17['cocina'].value_counts(dropna=False)

cocina
NaN                                                 9160
Cocina independiente                                 323
Cocina amueblada                                     190
Independiente                                        131
Cocina abierta                                        72
                                                    ... 
Con tendedero que se puede incluir en la cocicna       1
Grande                                                 1
Sin amueblar                                           1
Amplia y completamente amueblada                       1
amueblada equipada                                     1
Name: count, Length: 96, dtype: int64

In [2161]:
df17['cocina'].unique()

array([nan, 'Cocina abierta', 'Cocina independiente',
       'Independiente y equipada', 'Cocina americana', 'Cocina equipada',
       'Cocina amueblada', 'Amueblada sin electrodomésticos',
       'Amueblada con electrodomésticos', 'Americana',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente',
       'Independiente con office', 'Amueblada', 'Equipada', 'Amplia',
       'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Gran cocina abierta con isla', 'Moderna', 'Individual',
       'Hall de entrada', 'Reformada', 'Completamente equipada',
       'Totalmente equipada',
       'Abierta al comedor, completamente equipada',
       'En office amplia completamente equipada',
       'Cocina amueblada, sin aparatos', 'Con tendedero', 'Con terraza',
       'Dos cocinas, una en cada planta',
       'Amueblada con electrodomésticos, caldera nueva',
       'No incluye 

In [2162]:
df17.query("amueblado=='Sólo cocina amueblada' and cocina.notnull()")[['amueblado','cocina']]

,amueblado,cocina
492,Sólo cocina amueblada,Cocina abierta
1279,Sólo cocina amueblada,Cocina independiente
1748,Sólo cocina amueblada,Cocina independiente
1799,Sólo cocina amueblada,Cocina independiente
2823,Sólo cocina amueblada,Cocina independiente


In [2163]:
#pongo el valor de cocina_1 en cocina y elimino el campo cocina_1
#df17.loc[df17['cocina_1'].notnull(), 'cocina'] = df17['cocina_1']
#el campo cocina_equipada que aparece en redpiso lo integro en cocina y luebo elimino el campo
#df17.loc[df17['cocina_equipada']==1, 'cocina'] = 'Cocina equipada'
df17.loc[df17['cocina']=='Francesa', 'cocina'] = 'independiente y amueblada'
df17.loc[(df17['amueblado'] == 'Sólo cocina amueblada') & (df17['cocina'].notnull()), 'cocina'] = df17['cocina'] + ' amueblada'
df17.loc[df17['cocina']=='Equipafda americana', 'cocina'] = 'Equipada americana'

#df17.drop(columns=['cocina_1'], inplace=True)
#df17.drop(columns=['cocina_equipada'], inplace=True)


In [2164]:
# Aplica la función de agrupación
df17['grupo_cocina'] = df17['cocina'].apply(agrupar_cocinas)

print(df17[['cocina', 'grupo_cocina']])


               cocina     grupo_cocina
0                 NaN  sin_informacion
1      Cocina abierta          abierta
2                 NaN  sin_informacion
3                 NaN  sin_informacion
4                 NaN  sin_informacion
...               ...              ...
10198             NaN  sin_informacion
10199             NaN  sin_informacion
10200             NaN  sin_informacion
10201             NaN  sin_informacion
10202             NaN  sin_informacion

[10203 rows x 2 columns]


In [2165]:
df17.query("cocina.notnull()")['cocina'].unique()

array(['Cocina abierta', 'Cocina independiente',
       'Cocina abierta amueblada', 'Cocina independiente amueblada',
       'Independiente y equipada', 'Cocina americana', 'Cocina equipada',
       'Cocina amueblada', 'Amueblada sin electrodomésticos',
       'Amueblada con electrodomésticos', 'Americana',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente',
       'Independiente con office', 'Amueblada', 'Equipada', 'Amplia',
       'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Gran cocina abierta con isla', 'Moderna', 'Individual',
       'Hall de entrada', 'Reformada', 'Completamente equipada',
       'Totalmente equipada',
       'Abierta al comedor, completamente equipada',
       'En office amplia completamente equipada',
       'Cocina amueblada, sin aparatos', 'Con tendedero', 'Con terraza',
       'Dos cocinas, una en cada planta',
       'Amue

In [2166]:
df17.query("amueblado=='Sólo cocina amueblada'").count()

title                 69
url                   69
precio_anterior        0
descuento              0
EUR/m2                68
                      ..
orientacion_oeste      0
orientacion_sur        0
num_garajes            1
clasificacion_aire    69
grupo_cocina          69
Length: 91, dtype: int64

In [2167]:
df17['amueblado'].value_counts(dropna=False)

amueblado
 NaN                       8955
 1.0                        575
 Sí                         274
1.00                        159
 No                         158
 Sólo cocina amueblada       69
 Parcialmente amueblado      13
Name: count, dtype: int64

In [2168]:
## interior, exterior
df17['interior'].value_counts(dropna=False)

interior
NaN     7059
0.00    3123
1.00      21
Name: count, dtype: int64

In [2169]:

df17['exterior'].value_counts(dropna=False)

exterior
NaN     4628
0.00    3008
1.00    2567
Name: count, dtype: int64

In [2170]:

df17['interior_y_exterior'].value_counts(dropna=False)

interior_y_exterior
NaN     7061
0.00    3134
1.00       8
Name: count, dtype: int64

In [2171]:

df17.query("interior.isnull() & exterior.isnull() & interior_y_exterior.isnull()")["web"].unique()

array(['pisos.com', 'redpiso', 'pisosmadrid'], dtype=object)

## piscina

In [2172]:


df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     10143
1.00       60
Name: count, dtype: int64

In [2173]:
df17.loc[df17['url'] == 'https://www.indomio.es/anuncios/95261315/', 'piscina'] = 'comunitaria'

In [2174]:
df17['piscina'].value_counts(dropna=False)

piscina
 NaN                                          5986
 0                                            2744
0.00                                           750
 1                                             397
 Comunitaria                                   139
1.00                                           108
 Propia                                         76
 comunitaria                                     1
 1 piscina de adultos y 1 piscina de niños       1
 Con salorium                                    1
Name: count, dtype: int64

In [2175]:
df17['piscina'] = df17['piscina'].astype(str).str.lower().apply(unidecode).str.strip().replace('nan', None)

In [2176]:
df17['piscina']=df17['piscina'].replace('0.0','sin piscina')
df17['piscina']=df17['piscina'].replace('0','sin piscina')

In [2177]:
df17['piscina'].value_counts(dropna=False)

piscina
None                                         5986
sin piscina                                  3494
1                                             397
comunitaria                                   140
1.0                                           108
propia                                         76
1 piscina de adultos y 1 piscina de ninos       1
con salorium                                    1
Name: count, dtype: int64

In [2178]:
# Aplicar la lógica solo a los valores nulos en 'tiene_jardin'
df17['tiene_piscina'] = df17['tiene_piscina'].where(
    pd.notnull(df17['tiene_piscina']),  # Mantener valores no nulos existentes
    df17['piscina'].apply(lambda x: 1 if pd.notnull(x) and x != 'sin piscina' else (0 if x == 'sin piscina' else None))
)



In [2179]:
df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     5933
0.00    3491
1.00     779
Name: count, dtype: int64

In [2180]:
# Aplicar condiciones a la columna 'piscina'
df17['piscina'] = df17.apply(
    lambda x: 'privada' if pd.isnull(x['piscina']) and x['tiene_piscina'] == 1 and x['tipologia'] == 'Chalet' else
              ('comunitaria' if pd.isnull(x['piscina']) and x['tiene_piscina'] == 1 and x['tipologia'] != 'Chalet' else x['piscina']),
    axis=1
)



In [2181]:
df17['piscina'].value_counts(dropna=False)

piscina
None                                         5933
sin piscina                                  3494
1                                             397
comunitaria                                   184
1.0                                           108
propia                                         76
privada                                         9
1 piscina de adultos y 1 piscina de ninos       1
con salorium                                    1
Name: count, dtype: int64

In [2182]:
# Actualizar la columna 'tiene_piscina' donde 'piscina' es igual a 'sin piscina'
df17.loc[df17['piscina'] == 'sin piscina', 'tiene_piscina'] = 0




In [2183]:
df17.query("piscina=='sin piscina' & tiene_piscina==1")

,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,calefaccion,antiguedad,num_plantas,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,nuda_propiedad,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,€_comunidad_mes,acceso_discapacitado,orientacion,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga,num_visitas,fecha_publicacion,titularanuncio,num_pisos,puerta_seguridad,zonas_verdes,gastos_comunidad,tiene_piscina,tipo_fachada,agua_caliente,consumo_energia,nombre_oficina,letrace,portero,orientacion_este,orientacion_oeste,orientacion_sur,num_garajes,clasificacion_aire,grupo_cocina


## porton electrico, puerta blindada

In [2184]:

## portn_electrico,puerta_blindad

columnas = ["porton_electrico", "puerta_blindada", "alarma","videoportero","descripcion_larga"]
porcentaje_resultados = porcentaje_nulos(df17, columnas)


# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en porton_electrico: 69.21%
Porcentaje de nulos en puerta_blindada: 69.13%
Porcentaje de nulos en alarma: 68.23%
Porcentaje de nulos en videoportero: 66.44%
Porcentaje de nulos en descripcion_larga: 18.88%


In [2185]:

df17['porton_electrico'].value_counts(dropna=False)

porton_electrico
NaN     7061
0.00    3126
1.00      16
Name: count, dtype: int64

In [2186]:

df17['puerta_blindada'].value_counts(dropna=False)

puerta_blindada
NaN                                                   7053
0                                                     3050
1                                                       92
Acorazada                                                2
Tiene 3 puertas de acceso                                1
Muy cerca centro comercial las rosas, metro y bus.       1
Puerta acorazada                                         1
Bombillo seguridad multilock                             1
Puerta a la calle                                        1
Puerta de entrada blindada                               1
Name: count, dtype: int64

In [2187]:

df17['alarma'].value_counts(dropna=False)

alarma
 NaN                                              6961
 0                                                3100
1.00                                                48
 1                                                  42
 Portero físico                                     12
 vigilancia 24h, con cámaras de seguridad            5
 Conserje                                            4
 con cámaras de seguridad                            3
 Servicio de portería                                2
 Alarma                                              2
 Portero fisico                                      2
 24h                                                 2
 24 horas                                            2
 Alarma y cámaras                                    1
 Alarma instalada.                                   1
 Conserje 24 horas                                   1
 Circuito de video vigilancia en zonas comunes       1
 Cámaras de vigilancia en portal                     1
 Po

In [2188]:
# Actualizar la columna 'portero' únicamente donde 'alarma' contiene "portero"
df17['portero'] = df17.apply(
    lambda fila: 1 if "portero" in str(fila['alarma']).lower() else fila['portero'], axis=1
)

df17['portero'] = df17.apply(
    lambda fila: 1 if "conserje" in str(fila['alarma']).lower() else fila['portero'], axis=1
)

df17['portero'] = df17.apply(
    lambda fila: 1 if "vigilancia" in str(fila['alarma']).lower() else fila['portero'], axis=1
)

df17['alarma'] = df17.apply(
    lambda fila: 1 if "alarma" in str(fila['alarma']).lower() else fila['alarma'], axis=1
)

In [2189]:
df17.columns

Index(['title', 'url', 'precio_anterior', 'descuento', 'EUR/m2', 'm2_constr',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona', 'calle',
       'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia', 'estado',
       'amueblado', 'planta', 'dormitorios', 'banos', 'balcon', 'terraza',
       'garaje', 'trastero', 'ascensor', 'calefaccion', 'antiguedad',
       'num_plantas', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'nuda_propiedad', 'tipo_inmueble',
       'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpinteria_exterior_vidrio/metal', 'chimenea', 'cocina',
       'fibra_optica', 'exterior', 'interi

In [2190]:

df17.query("alarma==1 | puerta_blindada==1")[['precio','alarma','puerta_blindada']]

,precio,alarma,puerta_blindada
3373,1050000,1,NaN
3374,1985000,1,NaN
4331,225000,1,NaN
5869,145000,1,NaN
8715,81500,1.00,NaN
8754,115000,1.00,NaN
9297,925000,1.00,NaN
9351,419900,1.00,NaN
9376,229000,1.00,NaN
9399,450000,1.00,NaN


## descripcion_larga

€_comunidad_mes

In [2191]:
df17['€_comunidad_mes'].value_counts(dropna=False)

€_comunidad_mes
NaN    9676
50       23
60       22
100      17
70       15
       ... 
270       1
9         1
39        1
294       1
108       1
Name: count, Length: 204, dtype: int64

In [2192]:
df17['gastos_comunidad'].value_counts(dropna=False)

gastos_comunidad
NaN     10169
50          8
45          4
40          3
69          1
2110        1
500         1
550         1
380         1
60          1
236         1
38          1
30          1
35          1
150         1
74          1
65          1
110         1
100         1
180         1
20          1
99          1
850         1
Name: count, dtype: int64

In [2193]:
df17['gastos_comunidad'] = df17.apply(
    lambda fila: fila['€_comunidad_mes'] if pd.isna(fila['gastos_comunidad']) else fila['gastos_comunidad'], axis=1
)

In [2194]:
df17=df17.drop(columns='€_comunidad_mes')

In [2195]:
df17.shape

(10203, 90)

In [2196]:
#En redpiso los valores hay que dividirlos entre 100 al procesarlo se quito la coma que tenia en 20,00 ahora aparece con 2000
#pisos madrid y no tiene gastos de comunidad

# Sustituir las palabras 'Comunidad', '€', y '/mes' por vacío en 'gastos_comunidad'
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('Comunidad', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('€', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('/mes', '')
# Eliminar espacios en blanco adicionales
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.strip()
 #Convertir solo los valores no nulos de 'gastos_comunidad' a tipo float
df17.loc[df17['gastos_comunidad'].notnull(), 'gastos_comunidad'] = df17.loc[df17['gastos_comunidad'].notnull(), 'gastos_comunidad'].astype(float)
# Dividir el valor de 'gastos_comunidad' entre 100 cuando 'web' es igual a 'redpiso'
df17.loc[(df17['web'] == 'redpiso') & (df17['gastos_comunidad'].notnull()), 'gastos_comunidad'] /= 100


In [2197]:
df17.gastos_comunidad.unique()

array(['100', '123', nan, '55', '60', '258', '50', '92', '27', '90',
       '132', '128', '113', '800', '180', '1.300', '65', '625', '34',
       '150', '308', '146', 'Ningún gasto de comunidad', '310', '350',
       '110', '96', '120', '210', '70', '1.000', '250', '84', '247', '3',
       '56', '450', '40', '200', '75', '190', '166', '15', '45', '228',
       '72', '9', '294', '30', '160', '430', '95', '104', '63', '13',
       '85', '119', '300', '168', '340', '99', '164', '130', '241', '167',
       '5', '33', '111', '420', '206', '500', '10', '14', '540', '750',
       '148', '147', '386', '17', '76', '230', '205', '46', '47', '400',
       '32', '12', '36', '82', '155', '303', '207', '486', '83', '80',
       '66', '98', '178', '140', '52', '64', '54', '42', '29', '88', '61',
       '1.600', '440', '170', '105', '142', '255', '125', '600', '324',
       '67', '269', '285', '38', '39', '102', '270', '44', '103', '617',
       '552', '311', '457', '192', '327', '288', '316', '264', 

#############revisar

#Tengo que ir al origen para separar los valores de gastos de comunidad y de ibi

In [2198]:
df17['acceso_discapacitado'].value_counts(dropna=False)

acceso_discapacitado
 NaN                              9699
 Exteriores                        211
 0.0                               187
 1.0                                69
 Ver todas las características      25
1.00                                10
 Otros                               2
Name: count, dtype: int64

## acceso_discapacitados

In [2199]:

# Renombrar la columna 'acceso_discapacitado' a 'acceso_discapacidad'
df17.rename(columns={'acceso_discapacitado': 'acceso_discapacidad'}, inplace=True)

## orientacion

In [2200]:
df17["orientacion"].value_counts()

orientacion
Sur                                312
Este                               263
 Sur                               161
 Este                              150
Oeste                              148
Norte                              131
 Oeste                             125
Sureste                            105
 Norte                              80
 Sureste                            71
Suroeste                            59
 Suroeste                           59
 Noroeste                           53
Noreste                             52
Noroeste                            51
 Noreste                            50
sur                                 11
Medio día                           10
este                                 9
-1                                   9
norte                                4
Nordeste                             2
oeste                                2
A la montaña                         2
sur_este                             2
Sur en la ter

In [2201]:
# Eliminar 'Orientacion' y limpiar espacios adicionales
df17['orientacion'] = df17['orientacion'].str.replace('Orientacion', '').str.strip()

# Sustituir ',' y '-' por '_' y poner los valores en minúsculas en la columna 'orientacion'
df17['orientacion'] = df17['orientacion'].str.replace(',', '_').str.replace('/', '_').str.replace('|', '_').str.replace('y', '_').str.replace('-', '_').str.lower()

# Eliminar espacios adicionales entre palabras
df17['orientacion'] = df17['orientacion'].str.replace(r'\s+', '_', regex=True).str.strip()
df17['orientacion'] = df17['orientacion'].str.replace('__', '_')

# Reemplazar múltiples valores en la columna 'orientacion' por np.nan
df17['orientacion'] = df17['orientacion'].replace({
    '__':'_',
    '_1': np.nan,
    'medio_día': np.nan,
    'sur_en_la_terraza__dormitorios': np.nan,
    'vistas_al_palacio_de_parcent.': np.nan,
    'a_la_montaña': np.nan,
    'varias':np.nan
})

df17['orientacion'] = df17['orientacion'].str.replace('__', '_')



In [2202]:
# Obtener los valores únicos de la columna 'orientacion' donde los valores no son nulos
df17.loc[df17['orientacion'].notnull(), 'orientacion'].unique()


array(['norte', 'sur_este', 'este', 'oeste', 'sur', 'norte_sur',
       'suroeste', 'sureste', 'noroeste', 'este_oeste', 'noreste',
       'sur_este_norte', 'este_noreste_suroeste', 'oeste_este',
       'norte_este', 'este_oeste_norte', 'sur_oeste', 'sudeste',
       'sur_norte', 'nordeste'], dtype=object)

In [2203]:


# Procesar la columna 'orientacion'
for index, value in df17["orientacion"].items():
    # hasta los nan los convierte a string, recordar luego cambiar 'nan' por nan de nuevo
    value_str = str(value)
    if re.search(r'\bnorte\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
    if re.search(r'\bsur\b', value_str):
        df17.at[index, "orientacion_sur"] = 1
    if re.search(r'\beste\b', value_str):
        df17.at[index, "orientacion_este"] = 1
    if re.search(r'\boeste\b', value_str):
        df17.at[index, "orientacion_oeste"] = 1
    if re.search(r'\bnoroeste\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
        df17.at[index, "orientacion_oeste"] = 1
    if re.search(r'\bsuroeste\b', value_str):
        df17.at[index, "orientacion_sur"] = 1
        df17.at[index, "orientacion_oeste"] = 1
    if re.search(r'\bsureste\b', value_str):
        df17.at[index, "orientacion_sur"] = 1
        df17.at[index, "orientacion_este"] = 1
    if re.search(r'\bnordeste\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
        df17.at[index, "orientacion_este"] = 1
    if re.search(r'\bsur_este\b', value_str):
        df17.at[index, "orientacion_sur"] = 1
        df17.at[index, "orientacion_este"] = 1


        

# Revisar las columnas orientacion_norte,_sur y orientacion_sur,_este
# y asegurarnos de que las combinaciones sean coherentes
if "orientacion_norte,_sur" in df17.columns:
    for index, value in df17["orientacion_norte,_sur"].items():
        if value == 1:
            df17.at[index, "orientacion_norte"] = 1
            df17.at[index, "orientacion_sur"] = 1

if "orientacion_sur,_este" in df17.columns:
    for index, value in df17["orientacion_sur,_este"].items():
        if value == 1:
            df17.at[index, "orientacion_sur"] = 1
            df17.at[index, "orientacion_este"] = 1


In [2204]:
# Filtrar filas donde todas las columnas relevantes no sean nulas
filtered_df = df17[
    df17["orientacion"].notnull() |
    df17["orientacion_norte"].notnull() |
    df17["orientacion_sur"].notnull() |
    df17["orientacion_este"].notnull() |
    df17["orientacion_oeste"].notnull()
    #df17["orientacion_norte,_sur"].notnull() |
    #df17["orientacion_sur,_este"].notnull()
]

# Mostrar las columnas deseadas
result = filtered_df[["orientacion", "orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]]
print(result)


      orientacion  orientacion_norte  orientacion_sur  orientacion_este  \
93          norte               1.00              NaN               NaN   
174      sur_este                NaN             1.00              1.00   
228          este                NaN              NaN              1.00   
310      sur_este                NaN             1.00              1.00   
361         oeste                NaN              NaN               NaN   
...           ...                ...              ...               ...   
9762          NaN                NaN              NaN              1.00   
9874          NaN                NaN              NaN               NaN   
9907          NaN                NaN             1.00               NaN   
9908          NaN                NaN             1.00               NaN   
10103         NaN                NaN             1.00               NaN   

       orientacion_oeste  
93                   NaN  
174                  NaN  
228               

In [2205]:
#eliminacion de las columnas orientacion,orientacion_sur,_este,orientacion_norte,_sur
df17.drop(columns=['orientacion'], inplace=True)
#df17.drop(columns=['orientacion_sur,_este'], inplace=True)
#df17.drop(columns=['orientacion_norte,_sur'], inplace=True)


In [2206]:
df17.query("orientacion_norte!=1 and orientacion_sur!=1  and orientacion_este!=1  and orientacion_oeste!=1 ").count()

title                 8400
url                   8400
precio_anterior         55
descuento               55
EUR/m2                6755
                      ... 
orientacion_sur          0
num_garajes             12
clasificacion_aire    8400
grupo_cocina          8400
orientacion_norte        0
Length: 90, dtype: int64

In [2207]:
# Poner NaN en las columnas si no cumplen con la condición
df17.loc[df17.query("orientacion_norte!=1 and orientacion_sur!=1 and orientacion_este!=1 and orientacion_oeste!=1").index, ["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]] = np.nan

# Asegurarse de que si alguna columna tiene valor 1, las demás sean 0
df17[["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]] = df17[["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]].fillna(0)

## web

In [2208]:
df17["web"].value_counts(dropna=False)

web
pisos.com      4728
Indomio.com    3142
redpiso        1460
pisosmadrid     873
Name: count, dtype: int64

In [2209]:
df17["fecha_descarga"].value_counts(dropna=False)

fecha_descarga
2024-11-20    3142
19/08/2024    2745
19/10/2024    1983
28/10/2024    1460
24/10/2024     873
Name: count, dtype: int64

## certificado_energetico

In [2210]:
df17["certificado_energetico"].value_counts(dropna=False)

certificado_energetico
NaN            4196
0.0            2254
No indicado    1832
Disponible     1310
1.0             611
Name: count, dtype: int64

In [2211]:
df17.columns.tolist()

['title',
 'url',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'consumoce_ano',
 'letra_ce',
 'emisiones_co2',
 'tipologia',
 'estado',
 'amueblado',
 'planta',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'calefaccion',
 'antiguedad',
 'num_plantas',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'nuda_propiedad',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 'chimenea',
 'cocina',
 'fibra_optica',
 'exterior',
 'interior',
 'interior_y_exterior',
 'piscina',
 'porton_electrico',


In [2212]:
df17.query("consumoce_ano.notnull() | letrace.notnull() | emisiones_co2.notnull()" )[ ["certificado_energetico",'consumoce_ano','letrace','emisiones_co2']]

,certificado_energetico,consumoce_ano,letrace,emisiones_co2
0,Disponible,"≥ 3,51",NaN,D
1,No indicado,desconocido,NaN,No indicado
2,No indicado,desconocido,NaN,No indicado
3,No indicado,desconocido,NaN,No indicado
4,Disponible,desconocido,NaN,No indicado
...,...,...,...,...
9298,NaN,NaN,NaN,34.00
9317,NaN,NaN,NaN,45.00
9318,NaN,NaN,NaN,86.00
9431,NaN,NaN,B,NaN


In [2213]:
df17["emisiones_co2"].value_counts(dropna=False)

emisiones_co2
 NaN                    6800
 No indicado            2027
 Inclasificable          289
 E                       122
 D                        97
                        ... 
 45.7 kg CO₂/m² añoE       1
 26.3 kg CO₂/m² añoD       1
 321 kg CO₂/m² añoE        1
 13 kg CO₂/m² añoC         1
86.00                      1
Name: count, Length: 350, dtype: int64

In [2214]:
df17["consumoce_ano"].unique()

array(['≥ 3,51', 'desconocido', '90', '258.5', '129', '284', '207', '145',
       '170', '77', '138', '120', '231.1', '224', '191', '50', '194.2',
       '216', '121', '85', '123', '399', '999', '41.2', '294', '300',
       '173', '186', '193', '400', '170.3', '172.2', '444', '176',
       '209.7', '24', '74', '164', '163', '222', '2', '328', '242', '78',
       '89', '87', '382', '342', '441', '767', '111.1', '312', '199',
       '407', '336.3', '132', '266', '55', '248', '38', '463', '188',
       '126', '414', '155', '460', '301', '44', '354', '285', '178',
       '272', '260', '324', '221', '200.8', '35', '40', '47', '158',
       '262', '130', '114.1', '212', '192', '68.5', '214', '117', '186.9',
       '245', '299', '204', '137.9', '166', '55.5', '160', '255', '189',
       '195', '263', '244', '135', '180', '110', '321', '225', '313',
       '200', '149', nan], dtype=object)

In [2215]:
# Asignar letrace a letra_ce solo cuando letra_ce es nulo y letrace no es nulo
df17.loc[df17['letra_ce'].isnull() & df17['letrace'].notnull(), 'letra_ce'] = df17['letrace']
df17=df17.drop(columns='letrace')

In [2216]:
#se encuentra un piso que han modificado los valores en la web
# debe tener https://www.indomio.es/anuncios/97003125/, Consumo de energia:57.9 kWh/m² año, letrace:B, e Emisiones de CO₂:11.08 kg CO₂/m² año B
# Asignar valores específicos si la URL coincide
for index, value in df17["url"].items():
    if value == "https://www.indomio.es/anuncios/97003125/":
        df17.at[index, "consumoce_ano"] = 57.9
        df17.at[index, "letra_ce"] = "B"
        df17.at[index, "emisiones_co2"] = "11.08 kg CO₂/m² año B"


In [2217]:
df17["letra_ce"].value_counts(dropna=False)

letra_ce
NaN    8280
E       572
e       332
D       248
F       174
G       134
d       126
C        96
A        48
f        46
c        46
B        40
g        28
a        18
b        15
Name: count, dtype: int64

revisar el codigo que trae el fichero scrapingindomio.csv , C:\Mas\Dataset_Proyecto\Ficheros_Ventas\Indomio del 12/12/2024 quiza estamos mirando solo mayusculas

#

In [2218]:
df17.query("consumoce_ano=='≥ 3,51'")[['url','letra_ce','emisiones_co2','fecha_descarga']]

,url,letra_ce,emisiones_co2,fecha_descarga
0,https://www.indomio.es/anuncios/75404850/,F,D,2024-11-20
11,https://www.indomio.es/anuncios/83875795/,F,Pendiente,2024-11-20
12,https://www.indomio.es/anuncios/84201619/,G,Exento,2024-11-20
13,https://www.indomio.es/anuncios/84201619/,G,Exento,2024-11-20
87,https://www.indomio.es/anuncios/92109663/,G,Pendiente,2024-11-20
196,https://www.indomio.es/anuncios/93908759/,G,Pendiente,2024-11-20
197,https://www.indomio.es/anuncios/93908771/,G,Pendiente,2024-11-20
302,https://www.indomio.es/anuncios/94711699/,G,Pendiente,2024-11-20
313,https://www.indomio.es/anuncios/94729483/,G,E,2024-11-20
351,https://www.indomio.es/anuncios/94950781/,G,D,2024-11-20


CONSUMO Y LETRA
A: Menos de 50 kWh/m²/año
B: 51-90 kWh/m²/año
C: 91-150 kWh/m²/año
D: 151-230 kWh/m²/año
E: 231-330 kWh/m²/año
F: 331-450 kWh/m²/año
G: Más de 450 kWh/m²/año

En el caso de las emisiones:
A: Menos de 10 kg CO2/m²/año
B: 10-20 kg CO2/m²/año
C: 20-35 kg CO2/m²/año
D: 35-55 kg CO2/m²/año
E: 55-75 kg CO2/m²/año
F: 75-100 kg CO2/m²/año
G: Más de 100 kg CO2/m²/año

In [2219]:
df17.columns

Index(['title', 'url', 'precio_anterior', 'descuento', 'EUR/m2', 'm2_constr',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona', 'calle',
       'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia', 'estado',
       'amueblado', 'planta', 'dormitorios', 'banos', 'balcon', 'terraza',
       'garaje', 'trastero', 'ascensor', 'calefaccion', 'antiguedad',
       'num_plantas', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'nuda_propiedad', 'tipo_inmueble',
       'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpinteria_exterior_vidrio/metal', 'chimenea', 'cocina',
       'fibra_optica', 'exterior', 'interi

In [2220]:
#renombrar las columnas que hay de orientacion para integranlas con orientacion y sus valores

In [2221]:
#Revisar que filas no tiene descripcion_larga , deberian tenerlas todas
df17.query("descripcion_larga.isnull()")['web'].unique()

array(['Indomio.com', 'pisos.com', 'redpiso', 'pisosmadrid'], dtype=object)

## dormitorios

In [2222]:
df17['zonas_verdes'].value_counts(dropna=False)

zonas_verdes
NaN     10072
1.00      131
Name: count, dtype: int64

In [2223]:
porcentaje_nulos(df17,['zonas_verdes','nombre_oficina'])

{'zonas_verdes': 98.7160639027737, 'nombre_oficina': 85.69048319121826}

In [2224]:
df17.columns.tolist()

['title',
 'url',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'consumoce_ano',
 'letra_ce',
 'emisiones_co2',
 'tipologia',
 'estado',
 'amueblado',
 'planta',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'calefaccion',
 'antiguedad',
 'num_plantas',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'nuda_propiedad',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 'chimenea',
 'cocina',
 'fibra_optica',
 'exterior',
 'interior',
 'interior_y_exterior',
 'piscina',
 'porton_electrico',


In [2225]:
df17.query("consumoce_ano.notnull() or letra_ce.notnull() or emisiones_co2.notnull()")[['url','consumoce_ano','letra_ce','emisiones_co2','certificado_energetico','consumo_energia']]

,url,consumoce_ano,letra_ce,emisiones_co2,certificado_energetico,consumo_energia
0,https://www.indomio.es/anuncios/75404850/,"≥ 3,51",F,D,Disponible,NaN
1,https://www.indomio.es/anuncios/79178265/,desconocido,NaN,No indicado,No indicado,NaN
2,https://www.indomio.es/anuncios/81008864/,desconocido,NaN,No indicado,No indicado,NaN
3,https://www.indomio.es/anuncios/82164729/,desconocido,NaN,No indicado,No indicado,NaN
4,https://www.indomio.es/anuncios/82164739/,desconocido,E,No indicado,Disponible,NaN
...,...,...,...,...,...,...
9298,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-orense-cuatro-caminos-tetuan-madrid-madrid-RP2332023114339,NaN,NaN,34.00,NaN,138 KW
9317,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-casado-del-alisal-jeronimos-retiro-madrid-madrid-RP1792023115851,NaN,NaN,45.00,NaN,180 KW
9318,https://www.redpiso.es/inmueble/piso-en-venta-en-lista-salamanca-madrid-madrid-RP1642024124993,NaN,NaN,86.00,NaN,417 KW
9431,https://www.pisosmadrid.com.es/propiedad/3906133-piso-en-venta-en-madrid,NaN,B,NaN,NaN,NaN


In [2226]:
df17.query(" consumo_energia.notnull() ")[['url','consumoce_ano','letra_ce','emisiones_co2','consumo_energia']]

,url,consumoce_ano,letra_ce,emisiones_co2,consumo_energia
7872,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126007,NaN,NaN,18.0,111 KW
7873,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126427,NaN,NaN,24.0,145 KW
7876,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126442,NaN,NaN,17.0,104 KW
7882,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126423,NaN,NaN,14.0,88 KW
7883,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-alejandro-sanchez-opanel-carabanchel-madrid-madrid-RP512023115395,NaN,NaN,149.0,600 KW
...,...,...,...,...,...
9287,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-fresnedillas-fuentelarreina-fuencarral-el-pardo-madrid-madrid-RP722023118677,NaN,NaN,41.00,198 KW
9292,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-general-pardinas-58-lista-salamanca-madrid-madrid-RP1432024124685,NaN,NaN,68.00,326 KW
9298,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-orense-cuatro-caminos-tetuan-madrid-madrid-RP2332023114339,NaN,NaN,34.00,138 KW
9317,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-casado-del-alisal-jeronimos-retiro-madrid-madrid-RP1792023115851,NaN,NaN,45.00,180 KW


In [2227]:
df17["consumo_energia"].value_counts(dropna=False)

consumo_energia
NaN          9943
346 KW         14
222 KW          5
257 KW          4
204 KW          4
             ... 
309 KW          1
130 KW          1
261 KW          1
1.530 KW        1
417 KW          1
Name: count, Length: 171, dtype: int64

In [2228]:
df17.loc[df17['consumoce_ano'].isnull(), 'consumoce_ano'] = df17.loc[df17['consumoce_ano'].isnull(), 'consumo_energia'].apply(lambda x: extraer_numero(x) if isinstance(x, str) else None)

In [2229]:
columns_to_replace = ['consumoce_ano','letra_ce', 'emisiones_co2']
df17[columns_to_replace] = df17[columns_to_replace].replace(['desconocido', 'No indicado', 'Inclasificable','Pendiente','Exento','Cal energetica','Cal energetica Exento '], np.nan)

In [2230]:
df17["letra_ce"].value_counts(dropna=False)

letra_ce
NaN    8280
E       572
e       332
D       248
F       174
G       134
d       126
C        96
A        48
f        46
c        46
B        40
g        28
a        18
b        15
Name: count, dtype: int64

In [2231]:
df17.query("consumo_energia.notnull() ")[['url','consumoce_ano','letra_ce','emisiones_co2','consumo_energia','emisiones_co2']]

,url,consumoce_ano,letra_ce,emisiones_co2,consumo_energia,emisiones_co2
7872,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126007,111,NaN,18.0,111 KW,18.0
7873,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126427,145,NaN,24.0,145 KW,24.0
7876,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126442,104,NaN,17.0,104 KW,17.0
7882,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126423,88,NaN,14.0,88 KW,14.0
7883,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-alejandro-sanchez-opanel-carabanchel-madrid-madrid-RP512023115395,600,NaN,149.0,600 KW,149.0
...,...,...,...,...,...,...
9287,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-fresnedillas-fuentelarreina-fuencarral-el-pardo-madrid-madrid-RP722023118677,198,NaN,41.00,198 KW,41.00
9292,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-general-pardinas-58-lista-salamanca-madrid-madrid-RP1432024124685,326,NaN,68.00,326 KW,68.00
9298,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-orense-cuatro-caminos-tetuan-madrid-madrid-RP2332023114339,138,NaN,34.00,138 KW,34.00
9317,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-casado-del-alisal-jeronimos-retiro-madrid-madrid-RP1792023115851,180,NaN,45.00,180 KW,45.00


In [2232]:
df17["emisiones_co2"].value_counts(dropna=False)

emisiones_co2
 NaN                    9144
 E                       122
 D                        97
 F                        77
 C                        46
                        ... 
 45.7 kg CO₂/m² añoE       1
 26.3 kg CO₂/m² añoD       1
 321 kg CO₂/m² añoE        1
 13 kg CO₂/m² añoC         1
86.00                      1
Name: count, Length: 346, dtype: int64

In [2233]:
df17['emisiones_co2'] = df17['emisiones_co2'].str.replace('kg CO₂/m² año', '')
#df17['emisiones_co2'] = df17['emisiones_co2'].str.replace('kg', '')

In [2234]:
# Reemplazar 'NaN' con valores nulos de Pandas
df17['emisiones_co2'] = df17['emisiones_co2'].replace('NaN', np.nan)

# Extraer letra_emisiones y emisiones_ano
df17[['emisiones_ano', 'letra_emisiones']] = df17['emisiones_co2'].str.extract(r'(?:(\d+\.\d+|\d+)\s)?([A-Z])')

# Convertir emisiones_ano a float
df17['emisiones_ano'] = df17['emisiones_ano'].astype(float)

In [2235]:
df17['letra_emisiones'].value_counts(dropna=False)

letra_emisiones
NaN    9405
E       384
D       154
F       105
C        60
G        58
A        19
B        18
Name: count, dtype: int64

In [2236]:
df17['emisiones_ano'].value_counts(dropna=False)

emisiones_ano
NaN      9816
44.00      11
42.00       9
56.00       8
36.00       8
         ... 
56.90       1
49.80       1
58.10       1
18.00       1
86.00       1
Name: count, Length: 180, dtype: int64

In [2237]:
df17.query("consumoce_ano.notnull() and letra_ce.isnull()")[['url','consumoce_ano','letra_ce','emisiones_co2']]

,url,consumoce_ano,letra_ce,emisiones_co2
7872,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126007,111,NaN,18.0
7873,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126427,145,NaN,24.0
7876,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126442,104,NaN,17.0
7882,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126423,88,NaN,14.0
7883,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-alejandro-sanchez-opanel-carabanchel-madrid-madrid-RP512023115395,600,NaN,149.0
...,...,...,...,...
9287,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-fresnedillas-fuentelarreina-fuencarral-el-pardo-madrid-madrid-RP722023118677,198,NaN,NaN
9292,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-general-pardinas-58-lista-salamanca-madrid-madrid-RP1432024124685,326,NaN,NaN
9298,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-orense-cuatro-caminos-tetuan-madrid-madrid-RP2332023114339,138,NaN,NaN
9317,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-casado-del-alisal-jeronimos-retiro-madrid-madrid-RP1792023115851,180,NaN,NaN


In [2238]:
df17.query("consumo_energia.notnull() or letra_ce.notnull() or emisiones_co2.notnull()")[['url','consumoce_ano','letra_ce','emisiones_co2','letra_emisiones','emisiones_ano']]

,url,consumoce_ano,letra_ce,emisiones_co2,letra_emisiones,emisiones_ano
0,https://www.indomio.es/anuncios/75404850/,"≥ 3,51",F,D,D,NaN
4,https://www.indomio.es/anuncios/82164739/,NaN,E,NaN,NaN,NaN
11,https://www.indomio.es/anuncios/83875795/,"≥ 3,51",F,NaN,NaN,NaN
12,https://www.indomio.es/anuncios/84201619/,"≥ 3,51",G,NaN,NaN,NaN
13,https://www.indomio.es/anuncios/84201619/,"≥ 3,51",G,NaN,NaN,NaN
...,...,...,...,...,...,...
9298,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-orense-cuatro-caminos-tetuan-madrid-madrid-RP2332023114339,138,NaN,NaN,NaN,NaN
9317,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-casado-del-alisal-jeronimos-retiro-madrid-madrid-RP1792023115851,180,NaN,NaN,NaN,NaN
9318,https://www.redpiso.es/inmueble/piso-en-venta-en-lista-salamanca-madrid-madrid-RP1642024124993,417,NaN,NaN,NaN,NaN
9431,https://www.pisosmadrid.com.es/propiedad/3906133-piso-en-venta-en-madrid,None,B,NaN,NaN,NaN


In [2239]:
pd.set_option('display.max_rows', None)

In [2240]:
#eliminar las columnas consumo_energia,cal_energetica

In [2241]:
df17 = df17.drop(columns=['consumo_energia'])

In [2242]:
df17.head(1)

,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,calefaccion,antiguedad,num_plantas,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,nuda_propiedad,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,acceso_discapacidad,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga,num_visitas,fecha_publicacion,titularanuncio,num_pisos,puerta_seguridad,zonas_verdes,gastos_comunidad,tiene_piscina,tipo_fachada,agua_caliente,nombre_oficina,portero,orientacion_este,orientacion_oeste,orientacion_sur,num_garajes,clasificacion_aire,grupo_cocina,orientacion_norte,emisiones_ano,letra_emisiones
0,"Ático Calle de Núñez de Balboa 2, Recoletos, Madrid",https://www.indomio.es/anuncios/75404850/,NaN,NaN,20.00,100.00,Salamanca,4.00,Recoletos,41.00,Recoletos,calle de nunez de balboa 2,"≥ 3,51",F,D,Ático,Para reformar,Sólo cocina amueblada,1,1.00,1,1.00,1.00,2 en garaje privado,1.00,1.00,Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lujo,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,1.00,1.00,0.00,0.00,1,0.00,1,0,1.00,1.0,2000000,Indomio.com,2024-11-20,NaN,NaN,28001.00,NaN,NaN,NaN,Disponible,NaN,-Atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test atico test,NaN,NaN,NaN,NaN,NaN,NaN,100,1.00,NaN,NaN,NaN,Portero media jornada,0.00,0.00,0.00,NaN,Desconocido,sin_informacion,0.00,NaN,D


In [2243]:
# df17[['consumo_energia', 'cal_energetica','letrace','letra_emisiones']]

In [2244]:
# df17.query("certificado_energetico.notnull() and certificado_energetico!='No indicado'")[['consumo_energia', 'consumoce_ano','emisiones_ano','letrace','letra_emisiones','certificado_energetico']]

In [2245]:
# df17.query("certificado_energetico.notnull() and certificado_energetico!='No indicado'")[['consumo_energia', 'consumoce_ano','emisiones_ano','letrace','letra_emisiones','certificado_energetico']]

In [2246]:
#df17.to_csv("pisoscompleto.csv",index='False')

# Lectura pisoscompleto

In [2247]:
#df17 = pd.read_csv("pisoscompleto.csv", index_col=False)

In [2248]:


# Palabras clave para clasificar
central_keywords = ["central", "comunitaria", "comunitario"]
individual_keywords = ["individual", "particular", "chimenea", "bomba", "termo", "climatizacion"]
electric_keywords = ["electrica", "bomba", "climatización", 'termo electrico','climatizacion']
gas_keywords = ["gas", "propano", "g n", "g/n"]  # Asegurarse de que 'g n' y 'g/n' estén bien escritos
gasoil_keywords = ["gasóil","gasoil",'gasoil',"Gasoil"]  # Añadir "gasoil" aquí
renovables_keywords = ["aerotermia", "geotermia", "chimenea"]
suelo_radiante_keywords=["suelo_radiante", "suelo"]
#radiadores_keywords=["radiadores"]
aire_keywords=["bomba","climatización",'climatizacion','conducto']
desconocido_keywords=['chimenea', 'termo electrico']
#lo mas extendido son los radiadores por eso despues de pasar , suelo radiante y aire pongo a todos radiadores salvo a chimenea y termo electrico que no lo se
radiadores_keywords=['radiadores','Gas natural', 'G/n', 'Gas particular', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Comunitario', 'Individual con gas natural', 'Individual, de gasóil', 'Comunitaria', 'Calefaccion Propano', 'Calefaccion Gas Natural', 'Central, por aire, de gas', 'Gas natural y biomasa', 'Centralizada', 'Eléctrica', 'Calefaccion Individual', 'Calefaccion individual','Contadores individuales', '1.0', 'Individual, alimentación eléctrica', 'Aerotermia', 'Individual - gas natural', 'Individual', 'Calefaccion Aerotermia', 'Gasoil', 'Central individualizada', 'Individual, por aire, alimentación eléctrica', 'Geotermia y aerotermia', 'Calefaccion comunitaria', 'Calefaccion Central', 'Central, de gas', 'Calefaccion Electrica', 'Calefacción central individualizada', 'Central, de gasóil', 'Individual, de gas', 'Central']

# Inicializar conjuntos para valores únicos
central = set()
individual = set()
electrica = set()
gas = set()
gsoil = set()
renovables = set()
suelo_radiante = set()
radiadores = set()
aire = set()
sin_clasificar = set()
sin_clasificar_energia = set()
sin_clasificar_medio = set()
radiadores=set()
suelo_radiante=set()


# Clasificar categorías
for categoria in df17["calefaccion"]:
    categoria_normalized = normalize_text(str(categoria))  # Normalizar texto



    # Clasificación por tipo (central o individual)
    if any(keyword in categoria_normalized for keyword in central_keywords):
        central.add(categoria)
    elif any(keyword in categoria_normalized for keyword in individual_keywords):
        individual.add(categoria)
    else:
        sin_clasificar.add(categoria)

    # Clasificación por energía (eléctrica, gas, gasoil)
    if any(keyword in categoria_normalized for keyword in electric_keywords):
        electrica.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gasoil_keywords):
        gsoil.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gas_keywords):
        gas.add(categoria)


    elif any(keyword in categoria_normalized for keyword in renovables_keywords):
        renovables.add(categoria)
    else:
        sin_clasificar_energia.add(categoria)

    # Clasificación por medio (suelo radiante, radiadores, aire)


    if any(keyword in categoria_normalized for keyword in aire_keywords):
        aire.add(categoria)
    elif any(keyword in categoria_normalized for keyword in suelo_radiante_keywords):
        suelo_radiante.add(categoria)
    elif any(keyword in categoria_normalized for keyword in desconocido_keywords):
        sin_clasificar_medio.add(categoria)
  #  else: any(keyword in categoria_normalized for keyword in radiadores_keywords):
    else:
        radiadores.add(categoria)


# Convertir los conjuntos a listas
central = list(central)
individual = list(individual)
sin_clasificar = list(sin_clasificar)
electrica = list(electrica)
gas = list(gas)
gasoil = list(gsoil)
renovables = list(renovables)
sin_clasificar_energia = list(sin_clasificar_energia)
aire=list(aire)
radiadores=list(radiadores)
suelo_radiante=list(suelo_radiante)
sin_clasificar_medio=list(sin_clasificar_medio)
# Imprimir resultados
print("Central:", central)
print("Individual:", individual)
print("No clasificado:", sin_clasificar)
print("Electrica:", electrica)
print("Gas:", gas)
print("Gasoil:", gasoil)
print("Renovables:", renovables)
print("No clasificado en energía:", sin_clasificar_energia)
print("Suelo Radiante:", suelo_radiante)
print("Radiadores:", radiadores)
print("Aire:", aire)
print("No clasificado en medio:", sin_clasificar_medio)


Central: ['Calefaccion comunitaria', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Calefacción central individualizada', 'Central, por radiadores', 'Central individualizada', 'Central, de gas', 'Central, de gasóil', 'Central, por radiadores, de gas', 'Central, por radiadores, de gasóil', 'Centralizada', 'central', 'Comunitaria', 'Central, suelo radiante', 'Central, por aire, de gas', 'central por gas', 'Central', 'Central, suelo radiante, de gas', 'Comunitario']
Individual: ['Individual - gas natural', 'Bomba frio/calor', 'Chimenea', 'Gas particular', 'Individual, por radiadores, de gasóil', 'Por climatizacion', 'Individual, alimentación eléctrica', 'Individual, por radiadores', 'Individual, suelo radiante, de gas', 'Bomba frio-calor', 'Individual, de gas', 'Individual, por radiadores, de gas', 'Individual, por aire, por bomba de calor', 'Contadores individuales', 'Climatizacion', 'Individual, suelo radiante', 'Individual, de gasóil', 'indivi

In [2249]:
# Listas proporcionadas
listas = {
    'Central': ['Calefaccion comunitaria', 'Agua caliente y calefacción central con contador i', 'Central', 'Calefaccion Central', 'Calefacción central individualizada', 'Central, por radiadores, de gas', 'Comunitario', 'Central, por aire, de gas', 'Central, por radiadores', 'Comunitaria', 'Central individualizada', 'Central, de gas', 'Central, de gasóil', 'Centralizada', 'Central, por radiadores, de gasóil', 'Central, suelo radiante, de gas', 'Central, suelo radiante', 'Central, alimentación eléctrica'],
    'Individual': ['Gas particular', 'Climatizacion', 'Individual, suelo radiante', 'Calefaccion individual', 'Contadores individuales', 'climatizacion', 'Individual, por radiadores', 'Individual, por radiadores, de gasóil', 'Chimenea', 'Individual, alimentación eléctrica', 'Bomba frio-calor', 'Termo electrico', 'Individual, por aire, por bomba de calor', 'Individual, por radiadores, de gas', 'Individual con gas natural', 'Individual, suelo radiante, de gas', 'Bomba de calor', 'Individual, por aire, alimentación eléctrica', 'Bomba frio/calor', 'Individual - gas natural', 'Bomba frio calor', 'Individual, de gas', 'Calefaccion Individual', 'Individual, por radiadores, alimentación eléctrica', 'Por climatizacion', 'Individual', 'Individual, de gasóil'],
    'No clasificado': ['Calefaccion Propano', 'Calefaccion Electrica', 'Eléctrica', 'Suelo radiante agua', 'Conducto', 'Gasoil', 'Calefaccion Gas Natural', 'G/n', 'Geotermia y aerotermia', 'Calefaccion Aerotermia', '1.0', 'Calefaccion Suelo Radiante', 'Aerotermia', np.nan, 'Gas natural', 'Gas natural y biomasa', 'Suelo radiante'],
    'Electrica': ['Individual, por aire, por bomba de calor', 'Individual, por aire, alimentación eléctrica', 'Bomba frio calor', 'Calefaccion Electrica', 'Individual, por radiadores, alimentación eléctrica', 'Climatizacion', 'Eléctrica', 'Bomba frio/calor', 'Por climatizacion', 'climatizacion', 'Individual, alimentación eléctrica', 'Bomba de calor', 'Bomba frio-calor', 'Termo electrico', 'Central, alimentación eléctrica'],
    'Gas': ['Calefaccion Propano', 'Gas particular', 'Individual, por radiadores, de gas', 'Individual, de gas', 'Central, por radiadores, de gas', 'Individual con gas natural', 'Individual, suelo radiante, de gas', 'Central, por aire, de gas', 'Calefaccion Gas Natural', 'G/n', 'Individual - gas natural', 'Central, de gas', 'Central, suelo radiante, de gas', 'Gas natural', 'Gas natural y biomasa'],
    'Gasoil': ['Gasoil', 'Individual, por radiadores, de gasóil', 'Central, de gasóil', 'Central, por radiadores, de gasóil', 'Individual, de gasóil'],
    'Renovables': ['Calefaccion Aerotermia', 'Chimenea', 'Aerotermia', 'Geotermia y aerotermia'],
    'No clasificado en energía': ['Comunitario', 'Individual, suelo radiante', 'Calefaccion individual', 'Contadores individuales', 'Individual, por radiadores', 'Centralizada', 'Suelo radiante', 'Calefaccion Central', 'Calefacción central individualizada', 'Conducto', 'Central, suelo radiante', 'Agua caliente y calefacción central con contador i', 'Comunitaria', 'Central individualizada', 'Calefaccion Suelo Radiante', np.nan, 'Central, por radiadores', 'Calefaccion comunitaria', 'Central', 'Calefaccion Individual', 'Suelo radiante agua', '1.0', 'Individual'],
    'Suelo Radiante': ['Individual, suelo radiante, de gas', 'Suelo radiante agua', 'Individual, suelo radiante', 'Calefaccion Suelo Radiante', 'Central, suelo radiante, de gas', 'Central, suelo radiante', 'Suelo radiante'],
    'Radiadores': ['Gas particular', 'Calefaccion Electrica', 'Central, por radiadores, de gas', 'Comunitario', 'Central, por aire, de gas', 'Eléctrica', 'Gasoil', 'Geotermia y aerotermia', 'Calefaccion individual', 'Contadores individuales', 'Calefaccion Aerotermia', 'Individual, por radiadores', 'Individual, por radiadores, de gasóil', 'Centralizada', 'Individual, alimentación eléctrica', 'Calefaccion Central', 'Individual, por radiadores, de gas', 'Calefacción central individualizada', 'Individual con gas natural', 'G/n', 'Central, de gas', 'Central, de gasóil', 'Central, por radiadores, de gasóil', 'Agua caliente y calefacción central con contador i', 'Calefaccion Propano', 'Individual, por aire, alimentación eléctrica', 'Calefaccion Gas Natural', 'Comunitaria', 'Individual - gas natural', 'Central individualizada', np.nan, 'Central, por radiadores', 'Central, alimentación eléctrica', 'Calefaccion comunitaria', 'Central', 'Individual, de gas', 'Calefaccion Individual', 'Individual, por radiadores, alimentación eléctrica', '1.0', 'Individual', 'Aerotermia', 'Gas natural', 'Gas natural y biomasa', 'Individual, de gasóil'],
    'Aire': ['Individual, por aire, por bomba de calor', 'Bomba frio calor', 'Climatizacion', 'Conducto', 'Bomba frio/calor', 'Por climatizacion', 'climatizacion', 'Bomba de calor', 'Bomba frio-calor'],
    'No clasificado en medio': ['Termo electrico', 'Chimenea']
}

In [2250]:
# Definir las listas específicas para cada columna nueva
listas_tipo_1 = {k: listas[k] for k in ['Central', 'Individual']}
listas_tipo_2 = {k: listas[k] for k in ['Electrica', 'Gas', 'Gasoil','Renovables','No clasificado en energía']}
listas_tipo_3 = {k: listas[k] for k in ['Suelo Radiante', 'Radiadores', 'Aire', 'No clasificado en medio']}

# Aplicar la función a la columna calefaccion y crear tres nuevas columnas
df17['tipo_calefaccion'] = df17['calefaccion'].apply(lambda x: clasificar_calefaccion(x, listas_tipo_1))
df17['energia_calefaccion'] = df17['calefaccion'].apply(lambda x: clasificar_calefaccion(x, listas_tipo_2))
df17['instalacion_calefaccion'] = df17['calefaccion'].apply(lambda x: clasificar_calefaccion(x, listas_tipo_3))



In [2251]:
df17["amueblado"].value_counts(dropna=False)

amueblado
 NaN                       8955
 1.0                        575
 Sí                         274
1.00                        159
 No                         158
 Sólo cocina amueblada       69
 Parcialmente amueblado      13
Name: count, dtype: int64

In [2252]:
df17.columns.tolist()

['title',
 'url',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'consumoce_ano',
 'letra_ce',
 'emisiones_co2',
 'tipologia',
 'estado',
 'amueblado',
 'planta',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'calefaccion',
 'antiguedad',
 'num_plantas',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'nuda_propiedad',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 'chimenea',
 'cocina',
 'fibra_optica',
 'exterior',
 'interior',
 'interior_y_exterior',
 'piscina',
 'porton_electrico',


In [2253]:
df17.isna().sum()

title                                            0
url                                              0
precio_anterior                              10148
descuento                                    10148
EUR/m2                                        2349
m2_constr                                        5
distrito                                         7
cod_distrito                                     7
barrio                                          68
cod_barrio                                      68
zona                                          1586
calle                                         4837
consumoce_ano                                 9715
letra_ce                                      8280
emisiones_co2                                 9343
tipologia                                        0
estado                                        4615
amueblado                                     8955
planta                                        4649
dormitorios                    

In [2254]:
df17.isna().mean()*100

title                                        0.00
url                                          0.00
precio_anterior                             99.46
descuento                                   99.46
EUR/m2                                      23.02
m2_constr                                    0.05
distrito                                     0.07
cod_distrito                                 0.07
barrio                                       0.67
cod_barrio                                   0.67
zona                                        15.54
calle                                       47.41
consumoce_ano                               95.22
letra_ce                                    81.15
emisiones_co2                               91.57
tipologia                                    0.00
estado                                      45.23
amueblado                                   87.77
planta                                      45.57
dormitorios                                  2.22


In [2255]:
# Chequeo de valores en columna
df17["alarma"].value_counts()

alarma
0                                                3100
1                                                  52
1                                                  42
Portero físico                                     12
vigilancia 24h, con cámaras de seguridad            5
Conserje                                            4
con cámaras de seguridad                            3
Portero fisico                                      2
24 horas                                            2
Servicio de portería                                2
24h                                                 2
Conserje 24 horas                                   1
vigilancia 24h                                      1
Circuito de video vigilancia en zonas comunes       1
Cámaras de vigilancia en portal                     1
Camaras                                             1
Portería. Portero físico                            1
Portero fisico de dia.                              1
Puerta de seguridad  

In [2256]:
df17["orientacion_sur"].value_counts()

orientacion_sur
0.00    9420
1.00     783
Name: count, dtype: int64

In [2257]:
df17["orientacion_oeste"].value_counts()

orientacion_oeste
0.00    9705
1.00     498
Name: count, dtype: int64

In [2258]:
df17["orientacion_norte"].value_counts()

orientacion_norte
0.00    9882
1.00     321
Name: count, dtype: int64

In [2259]:
df17["orientacion_este"].value_counts()

orientacion_este
0.00    9600
1.00     603
Name: count, dtype: int64

In [2260]:
df17["puerta_seguridad"].value_counts()

puerta_seguridad
1.00    18
Name: count, dtype: int64

In [2261]:
df17["puerta_blindada"].value_counts()

puerta_blindada
0                                                     3050
1                                                       92
Acorazada                                                2
Tiene 3 puertas de acceso                                1
Muy cerca centro comercial las rosas, metro y bus.       1
Puerta acorazada                                         1
Bombillo seguridad multilock                             1
Puerta a la calle                                        1
Puerta de entrada blindada                               1
Name: count, dtype: int64

In [2262]:
df17["zonas_verdes"].value_counts()

zonas_verdes
1.00    131
Name: count, dtype: int64

In [2263]:
df17["codigo_Postal"].unique()

array([28001., 28042., 28045., 28016., 28020., 28002., 28015., 28021.,
       28024., 28006., 28004., 28035., 28031., 28053., 28013., 28010.,
       28028., 28046., 28014., 28037., 28027., 28055., 28012., 28008.,
       28011., 28019., 28003., 28009., 28007., 28039., 28033., 28032.,
       28044., 28041., 28043., 28017., 28018., 28029., 28023., 28038.,
       28034., 28047., 28026., 28048., 28025., 28040., 28030., 28005.,
          nan, 28049.])

In [2264]:
#df17.to_csv("pisoscompleto.csv",index='False')

# Lectura pisoscompleto

In [2265]:
#df17 = pd.read_csv("pisoscompleto.csv", index_col=False)

In [2266]:
dfadicionales= pd.read_csv('../02-datos-limpios/Datos_adicionales.csv',sep=";", index_col=False)

In [2267]:
dfadicionales.columns

Index(['cod_municipio', 'municipio', 'cod_distrito', 'distrito', 'cod_barrio',
       'barrio', 'codigo_Postal', 'm2_areasverdes', 'num_personas',
       'num_personas_hombres', 'num_personas_mujeres', 'total_transacciones',
       'viv_nuevas_transacciones', 'viv_usadas_transacciones', 'lineas_metro',
       'salidas_metro', 'estaciones_metroligero', 'estaciones_cercanias',
       'estacion_bus_urbano', 'num_lineas_buses',
       'Renta neta media por persona', 'Renta neta media por hogar',
       'Mediana de la renta por unidad de consumo',
       'Renta bruta media por persona', 'Renta bruta media por hogar',
       'Habitantes', 'Hogares', 'Tamaño medio del hogar',
       'Tamaño del hogar (Nº de personas en la vivienda)', 'hog_con_1',
       'hog_con_2', 'hog_con_3', 'hog_con_4', 'hog_con_5', 'hog_con_6',
       'hog_con_7', 'hog_con_8', 'hog_con_9', 'hog_con_10', 'hog_con_11',
       'hog_con_12', 'hog_con_13', 'hog_con_14', 'hog_con_15 y más',
       'Comercio_ServiciosMinorista

In [2268]:
#uno los datos adicionales a nivel de barrio para las columnas  codigo_Postal,	num_personas,	num_personas_hombres	,num_personas_mujeres	,total_transacciones,	viv_nuevas_transacciones,	viv_usadas_transacciones
# uno a nivel de distrito lineas_metro	salidas_metro	estaciones_metroligero	estaciones_cercanias	estacion_bus_urbano	num_lineas_buses	Renta neta media por persona	Renta neta media por hogar	Mediana de la renta por unidad de consumo	Renta bruta media por persona	Renta bruta media por hogar	Habitantes	Hogares	Tamaño medio del hogar	Tamaño del hogar (Nº de personas en la vivienda)	hog_con_1	hog_con_2	hog_con_3	hog_con_4	hog_con_5	hog_con_6	hog_con_7	hog_con_8	hog_con_9	hog_con_10	hog_con_11	hog_con_12	hog_con_13	hog_con_14	hog_con_15 y más


In [2269]:
df17.loc[df17['url'].str.contains('calle-narvaez'), ['cod_barrio', 'barrio']] = [34, 'Ibiza']
df17.loc[df17['url'].str.contains('calle-claudio-coello'), ['cod_barrio', 'barrio']] = [41, 'Recoletos']
df17.loc[df17['url'].str.contains('calle-maria-de-molina'), ['cod_barrio', 'barrio']] = [51, 'El Viso']
df17.loc[df17['url'].str.contains('calle-eloy-gonzalo'), ['cod_barrio', 'barrio']] = [73, 'Trafalgar']

df17['titular_anuncio'] = df17['descripcion_larga'].fillna('')
df17.loc[df17['titular_anuncio'].str.contains('Ibiza'), ['cod_barrio', 'barrio']] = [34, 'Ibiza']


# Reemplazar valores NaN en 'descripcion_larga' con una cadena vacía
df17['descripcion_larga'] = df17['descripcion_larga'].fillna('')
df17.loc[df17['descripcion_larga'].str.contains('Calle Mariano Benlliure'), ['cod_barrio', 'barrio','cod_distrito','distrito']] = [74, 'Almagro',7,'Chamberí']
df17.loc[df17['descripcion_larga'].str.contains('Calle Moquetas 11'), ['cod_barrio', 'barrio','cod_distrito','distrito','estado']] = [74, 'Almagro',7,'Chamberí','Obra nueva']
df17.loc[df17['descripcion_larga'].str.contains('calle de Castello'), ['cod_barrio', 'barrio']] = [74, 'Almagro']
df17.loc[df17['descripcion_larga'].str.contains('IBIZA'), ['cod_barrio', 'barrio']] = [34, 'Ibiza']

In [2270]:
df17['cod_barrio']

0        41.00
1       213.00
2        24.00
3        54.00
4        54.00
5        54.00
6        54.00
7        62.00
8        51.00
9        42.00
10       72.00
11      171.00
12      105.00
13      105.00
14       46.00
15       14.00
16       87.00
17       54.00
18      183.00
19      132.00
20       72.00
21       41.00
22       16.00
23       74.00
24       45.00
25       51.00
26       54.00
27       54.00
28       55.00
29       45.00
30       41.00
31       41.00
32       15.00
33       42.00
34       55.00
35       73.00
36       15.00
37       43.00
38       56.00
39       56.00
40       54.00
41       35.00
42       55.00
43       16.00
44       16.00
45       41.00
46       41.00
47       41.00
48       63.00
49       51.00
50      206.00
51       11.00
52      153.00
53      166.00
54       55.00
55       46.00
56       55.00
57       12.00
58       74.00
59       92.00
60       62.00
61       41.00
62       41.00
63       91.00
64       52.00
65      112.00
66       1

In [2271]:
df17["cod_barrio"].value_counts(dropna=False)

cod_barrio
15.00     449
42.00     442
16.00     351
46.00     337
41.00     334
11.00     279
14.00     267
12.00     255
81.00     234
34.00     228
45.00     210
13.00     185
74.00     174
104.00    143
54.00     143
51.00     139
152.00    134
52.00     132
62.00     123
73.00     123
55.00     122
132.00    118
92.00     117
136.00    117
201.00    111
114.00    104
43.00     103
171.00    100
44.00      95
35.00      94
166.00     93
183.00     92
131.00     89
76.00      86
61.00      86
75.00      86
63.00      86
116.00     85
115.00     84
207.00     80
56.00      80
71.00      79
112.00     78
181.00     78
134.00     73
93.00      73
117.00     73
163.00     73
25.00      71
102.00     70
107.00     69
135.00     69
NaN        68
175.00     67
83.00      67
113.00     66
97.00      65
151.00     63
72.00      63
22.00      62
133.00     60
66.00      59
182.00     58
164.00     56
103.00     56
172.00     55
153.00     54
174.00     52
194.00     52
31.00      50
84.00    

In [2272]:
# Convertir las columnas a enteros, ignorando los errores
df17['cod_distrito'] = pd.to_numeric(df17['cod_distrito']).astype('Int64')
df17['cod_barrio'] = pd.to_numeric(df17['cod_barrio']).astype('Int64')

In [2273]:
# Concatenar cod_barrio y cod_distrito si cod_barrio < 11
df17.loc[df17['cod_barrio'] < 11, 'cod_barrio'] =  df17['cod_distrito'].astype(str)+df17['cod_barrio'].astype(str)


In [2274]:
df17["m2_utiles"].value_counts(dropna=False)

m2_utiles
NaN            7941
 60              40
 50              33
 80              32
 66              30
90 m²            30
 65              29
 75              29
75 m²            26
 70              22
50 m²            22
60 m²            22
 40              22
80 m²            21
70 m²            21
 85              21
 51              20
 55              20
67 m²            20
65 m²            19
95 m²            19
 59              19
 62              19
 67              19
 57              18
 61              18
 78              17
 52              17
63 m²            17
 58              16
88 m²            16
105 m²           16
 68              16
100 m²           16
 74              15
 72              15
85 m²            15
 81              15
84 m²            15
98 m²            14
40 m²            14
 53              13
 90              13
 100             13
 71              13
82 m²            13
78 m²            13
45 m²            13
61 m²            13
 42       

In [2275]:
df17["m2_utiles"]=df17["m2_utiles"].str.replace("Metros","")
df17["m2_utiles"]=df17["m2_utiles"].str.replace("m²","")

In [2276]:
df17.drop_duplicates(subset=['url'], keep='first', inplace=True)


In [2277]:
df17['url'].nunique() == len(df17)

True

In [2278]:
df17.drop_duplicates(subset=['url'], keep='first', inplace=True)


In [2279]:
df17["url"].value_counts(dropna=False)

url
https://www.indomio.es/anuncios/75404850/                                                                                                                                                         1
https://www.pisos.com/comprar/piso-orcasitas-48404936035_100200/                                                                                                                                  1
https://www.pisos.com/comprar/piso-opanel28019-48347307178_100200/                                                                                                                                1
https://www.pisos.com/comprar/piso-opanel28019-48417820084_100200/                                                                                                                                1
https://www.pisos.com/comprar/piso-opanel28019-32547594015_100200/                                                                                                                                1
https://www.piso

Elimino la columna codigo porstal por que la voy a traer al hacer el merge con dfadicionales

In [2280]:
df17.shape

(10113, 94)

In [2281]:
df17.precio.describe()

count      10113.00
mean      887858.30
std      1160881.31
min        25000.00
25%       239000.00
50%       477500.00
75%      1080000.00
max     19950000.00
Name: precio, dtype: float64

In [ ]:
# Replace values in the 'portero' column
df17['portero'] = np.where(df17['portero'] == 'Portero media jornada', 0.5, df17['portero'])
df17['portero'] = np.where(df17['portero'] == 'Portero todo el día', 1, df17['portero'])
df17['portero'] = np.where(df17['portero'] == '1', 1, df17['portero'])

In [2282]:
# Eliminar el símbolo '%' y convertir a tipo numérico
df17['descuento'] = df17['descuento'].str.replace('%', '')
df17['descuento'] = pd.to_numeric(df17['descuento'].str.replace(',', '.'))
#df17["alarma"] = df17["alarma"].fillna(df17["sistema_alarma"])
#puerta de seguridad y puerta_blindada en una columna
df17["puerta_seguridad"] = df17["puerta_seguridad"].fillna(df17["puerta_blindada"])
df17=df17.drop(columns="puerta_blindada")

In [2283]:
df17["letra_ce"].value_counts(dropna=False)

letra_ce
NaN    8220
E       563
e       332
D       238
F       170
G       131
d       126
C        93
A        47
f        46
c        46
B        40
g        28
a        18
b        15
Name: count, dtype: int64

In [2284]:
# Aplicar la función a la columna consumo_ce solo cuando letrace es nulo y consumo_ce no es nulo
df17['letra_ce'] = df17.apply(lambda row: clasificar_consumo(row['consumoce_ano']) if pd.isnull(row['letra_ce']) and not pd.isnull(row['consumoce_ano']) else row['letra_ce'], axis=1)

In [2285]:
df17["m2_constr"].value_counts(dropna=False).sort_values(ascending=True)


m2_constr
120.54       1
893.00       1
491.00       1
88.44        1
105.54       1
120.56       1
373.00       1
445.00       1
490.00       1
441.00       1
334.00       1
267.00       1
379.00       1
463.00       1
1192.00      1
361.00       1
279.00       1
431.00       1
1775.00      1
577.00       1
901.00       1
528.00       1
14.00        1
781.00       1
367.00       1
561.00       1
103.91       1
375.00       1
666.00       1
566.00       1
934.00       1
386.00       1
601.00       1
541.00       1
620.00       1
409.00       1
377.00       1
461.00       1
16.00        1
10.00        1
421.00       1
468.00       1
530.00       1
730.00       1
484.00       1
352.00       1
752.00       1
419.00       1
444.00       1
358.00       1
79.95        1
518.00       1
405.00       1
347.00       1
1800.00      1
1582.00      1
838.00       1
287.00       1
662.00       1
479.00       1
605.00       1
326.00       1
436.00       1
422.00       1
288.00       1
700.00       1


In [2286]:
df17= df17[df17['tipologia'] != 'Negocio']
df17['tipologia'] = df17['tipologia'].replace('Venta', 'Piso')
# Actualizar los valores de 'distrito' y 'barrio' cuando 'cod_barrio' es 195
df17.loc[df17['cod_barrio'] == 195.00, ['distrito', 'barrio']] = ['Vicalvaro', 'Los Berrocales']



# Seleccionar solo las columnas que quieres añadir
#columns_to_add = ['cod_barrio', 'codigo_Postal', 'num_personas', 'num_personas_hombres', 'num_personas_mujeres', 'total_transacciones', 'viv_nuevas_transacciones', 'viv_usadas_transacciones']
#columns_to_add_d = ['cod_distrito', 'codigo_Postal', 'num_personas', 'num_personas_hombres', 'num_personas_mujeres', 'total_transacciones', 'viv_nuevas_transacciones', 'viv_usadas_transacciones']
columns_to_add = ['cod_barrio',  'num_personas', 'num_personas_hombres', 'num_personas_mujeres', 'total_transacciones', 'viv_nuevas_transacciones', 'viv_usadas_transacciones']
columns_to_add_d = ['cod_distrito',  'num_personas', 'num_personas_hombres', 'num_personas_mujeres', 'total_transacciones', 'viv_nuevas_transacciones', 'viv_usadas_transacciones']

# Filtrar el DataFrame adicional para mantener solo las columnas seleccionadas
df_adicional_filtered = dfadicionales[columns_to_add]
#df_adicional_filtered_d = dfadicionales[columns_to_add_d]
df_adicional_filtered_d = dfadicionales[dfadicionales['cod_barrio'].isna()][columns_to_add_d]

# Filtrar filas en df17 donde cod_barrio no es nulo y hacer el primer merge
df17_filtered = df17.dropna(subset=['cod_barrio'])
df_merged_barrio = pd.merge(df17_filtered, df_adicional_filtered, on='cod_barrio', how='left')

# Filtrar filas en df17 donde cod_barrio es nulo
df17_na = df17[df17['cod_barrio'].isna()]

# Realizar el merge con las filas filtradas usando cod_distrito
df_merged_na = pd.merge(df17_na, df_adicional_filtered_d, on='cod_distrito', how='left')

# Combinar los dos DataFrames resultantes en uno final
df_final = pd.concat([df_merged_barrio, df_merged_na], ignore_index=True)




In [2287]:
#df_final.columns.tolist()

In [2288]:
df17["cod_barrio"].isnull().sum()

68

df17.shape,df_merged_na.shape, df17_na.shape,df_adicional_filtered_d.shape,df_final.shape

In [2289]:
#ahora hago lo mismo para las que voy a hacer merge por distrito

In [2290]:
dfadicionales.columns

Index(['cod_municipio', 'municipio', 'cod_distrito', 'distrito', 'cod_barrio',
       'barrio', 'codigo_Postal', 'm2_areasverdes', 'num_personas',
       'num_personas_hombres', 'num_personas_mujeres', 'total_transacciones',
       'viv_nuevas_transacciones', 'viv_usadas_transacciones', 'lineas_metro',
       'salidas_metro', 'estaciones_metroligero', 'estaciones_cercanias',
       'estacion_bus_urbano', 'num_lineas_buses',
       'Renta neta media por persona', 'Renta neta media por hogar',
       'Mediana de la renta por unidad de consumo',
       'Renta bruta media por persona', 'Renta bruta media por hogar',
       'Habitantes', 'Hogares', 'Tamaño medio del hogar',
       'Tamaño del hogar (Nº de personas en la vivienda)', 'hog_con_1',
       'hog_con_2', 'hog_con_3', 'hog_con_4', 'hog_con_5', 'hog_con_6',
       'hog_con_7', 'hog_con_8', 'hog_con_9', 'hog_con_10', 'hog_con_11',
       'hog_con_12', 'hog_con_13', 'hog_con_14', 'hog_con_15 y más',
       'Comercio_ServiciosMinorista

#ahora añado las columnas que estan a nivel de distrito
#lineas_metro	salidas_metro	estaciones_metroligero	estaciones_cercanias	estacion_bus_urbano	num_lineas_buses	Renta neta media por persona	Renta neta media por hogar	Mediana de la renta por unidad de consumo	Renta bruta media por persona	Renta bruta media por hogar	Habitantes	Hogares	Tamaño medio del hogar	Tamaño del hogar (Nº de personas en la vivienda)	hog_con_1	hog_con_2	hog_con_3	hog_con_4	hog_con_5	hog_con_6	hog_con_7	hog_con_8	hog_con_9	hog_con_10	hog_con_11	hog_con_12	hog_con_13	hog_con_14	hog_con_15 y más
columns_to_add_d = ['cod_distrito','lineas_metro','salidas_metro','estaciones_metroligero','estaciones_cercanias','estacion_bus_urbano','num_lineas_buses','Renta neta media por persona','Renta neta media por hogar','Mediana de la renta por unidad de consumo','Renta bruta media por persona','Renta bruta media por hogar','Habitantes','Hogares','Tamaño medio del hogar','Tamaño del hogar (Nº de personas en la vivienda)','hog_con_1','hog_con_2','hog_con_3'	,'hog_con_4','hog_con_5','hog_con_6','hog_con_7','hog_con_8','hog_con_9','hog_con_10','hog_con_11',	'hog_con_12','hog_con_13','hog_con_14','hog_con_15 y más','Comercio_ServiciosMinoristas', 'Hosteleria_Restauracion',
       'Salud_Bienestar_Educacion', 'Servicios_Profesionales','Cultura_Entretenimiento','m2_areasverdes']

df_adicional_filtered_d = dfadicionales[dfadicionales['cod_barrio'].isna()][columns_to_add_d]

# Unir los DataFrames por 'cod_barrio'
df_final = pd.merge(df_final, df_adicional_filtered_d, on='cod_distrito', how='left')

In [2291]:
# Filtrar las filas que no se pueden convertir a float sin modificar la columna 'precio'
#df_final[~df_final['precio'].astype(str).str.replace(',', '').str.replace(' ', '').str.isnumeric()][["title","url","precio"]]



In [2292]:
df_final=df17

In [2293]:
df_final.shape

(10109, 93)

In [2294]:
df_final.columns

Index(['title', 'url', 'precio_anterior', 'descuento', 'EUR/m2', 'm2_constr',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona', 'calle',
       'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia', 'estado',
       'amueblado', 'planta', 'dormitorios', 'banos', 'balcon', 'terraza',
       'garaje', 'trastero', 'ascensor', 'calefaccion', 'antiguedad',
       'num_plantas', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'nuda_propiedad', 'tipo_inmueble',
       'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpinteria_exterior_vidrio/metal', 'chimenea', 'cocina',
       'fibra_optica', 'exterior', 'interi

In [2295]:
#df_final.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2', 'Unnamed: 0.3', 'Unnamed: 0.4'], inplace=True)


In [2296]:
#df_final.rename(columns={'tamaño del hogar (nº de personas en la vivienda)': 'tamano_hogar'}, inplace=True)

In [2297]:
#df_final.columns = df_final.columns.str.replace(' ', '_',regex=False)

In [2298]:
# Aplicar la lógica: valores no nulos y diferentes de 0.00 se convierten en 1
df_final['garaje'] = df_final['garaje'].apply(lambda x: 1 if pd.notna(x) and x!=0.00 else x)

In [2299]:
df_final.loc[df_final['num_garajes'] == 2, 'garaje'] = 1

In [2300]:
#df_final=df_final.drop(columns='num_garajes')

In [2301]:
df_final['garaje'].unique()

array([ 1., nan,  0.])

In [2302]:
df_final["interior_y_exterior"].value_counts(dropna=False)

interior_y_exterior
NaN     7057
0.00    3044
1.00       8
Name: count, dtype: int64

In [2303]:
# Asignar 1 a 'interior' y 'exterior' cuando 'interior_y_exterior' es igual a 1.00
df_final.loc[df_final['interior_y_exterior'] == 1.00, ['interior', 'exterior']] = 1


In [2304]:
#vamos a agrupar algunas categorias que están repetidas con acentos y sin acentos

df_final["tipologia"]=df_final["tipologia"].replace({'Ático': 'Atico', 'Dúplex': 'Duplex'})
df_final["distrito"]=df_final["distrito"].replace({'Puente de Vallecas': 'Puente De Vallecas'})


#Convertir los valores de 'letra_ce' a mayúsculas
df_final['letra_ce'] = df_final['letra_ce'].str.upper()
df_final['letra_emisiones'] = df_final['letra_ce'].str.upper()
df_final['estado'] = df_final['estado'].str.strip().str.title()



In [2305]:
# Rellenar los valores nulos de 'garajes' con los valores de 'num_garajes'
df_final['garaje'] = df_final['garaje'].fillna(df_final['num_garajes'])

In [2306]:
#Elimino las filas con cod_barrio nulo
# Eliminar filas donde 'cod_barrio' es nulo
df_final = df_final.dropna(subset=['cod_barrio'])
#eLIMINO LAS FILAS con M2_CONSTR nulo
df_final=df_final.dropna(subset=['m2_constr'])

In [2307]:
df_final.reset_index(drop=True, inplace=True)

In [2308]:
#columna 45 acceso_discapacidad, eliminar los valores
val=['Exteriores' 'Ver todas las características' 'Otros']
df_final['acceso_discapacidad'] = df_final['acceso_discapacidad'].replace(val, np.nan)
# Replace '.0' and similar patterns with empty string using regex
df_final['acceso_discapacidad'] = df_final['acceso_discapacidad'].astype(str).str.replace('.0', '', regex=True)
df_final['acceso_discapacidad'] = pd.to_numeric(df_final['acceso_discapacidad'], errors='coerce').astype('Int64') # Convert to numeric, handle errors, then to Int64

cols=['amueblado','puerta_seguridad','piscina','jardin']
for c in cols:
  df_final[c]=df_final[c].astype(str)

In [2309]:
cambios = {'3+': '5',  '.0': '','0.0':np.nan,'nan':np.nan}
# Reemplazar los valores '3+' por 5 en la columna 'banos'
df_final['banos'] = df_final['banos'].astype(str).replace(cambios).str.strip()


In [2310]:
#56 puerta_seguridad
cambios = {'Puerta acorazada': '1',  'Bombillo seguridad multilock': '1', 'Acorazada': '1','Puerta de entrada blindada':'1','Puerta a la calle':'1','1.0':'1'}
cambiosnan={'Tiene 3 puertas de acceso','Muy cerca centro comercial las rosas, metro y bus.'}
df_final['puerta_seguridad'] = df_final['puerta_seguridad'].astype(str).replace(cambios).str.strip()
df_final['puerta_seguridad'] = df_final['puerta_seguridad'].replace(cambiosnan,np.nan)
df_final['puerta_seguridad'] = pd.to_numeric(df_final['puerta_seguridad'], errors='coerce')
# Change the astype to pd.Int64Dtype() to handle NaN
df_final['puerta_seguridad'] = df_final['puerta_seguridad'].astype(pd.Int64Dtype())

In [2311]:
cambios = {'Más de 2': '4', 'Mas de 5': '7', '.0': ''}
df_final['armarios'] = df_final['armarios'].replace(cambios).str.strip()


# creacion dataframe EDA

In [2312]:
df_final.columns.tolist()

['title',
 'url',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'consumoce_ano',
 'letra_ce',
 'emisiones_co2',
 'tipologia',
 'estado',
 'amueblado',
 'planta',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'calefaccion',
 'antiguedad',
 'num_plantas',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'nuda_propiedad',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 'chimenea',
 'cocina',
 'fibra_optica',
 'exterior',
 'interior',
 'interior_y_exterior',
 'piscina',
 'porton_electrico',


In [2313]:
df_final["num_pisos"]=df_final["num_pisos"].replace('Pisos:','',regex=True)

In [2314]:
# Eliminar filas donde 'm2_constr' sea igual a 0
df_final = df_final.query("m2_constr != 0")

In [2315]:
# Aplicar la lógica
#df_final.loc[df_final['interior_y_exterior'] == 1, ['interior', 'exterior']] = 1
#df_final=df_final.drop(columns='interior_y_exterior')


In [2316]:
df_final.barrio.nunique()

132

In [2317]:
df_final["barrio"]=df_final["barrio"].replace("Niño Jesús",'Niño Jesus',regex=True)

In [2318]:
df_final.barrio.nunique()

131

In [2319]:
df_final["grupo_cocina"].unique()

array(['sin_informacion', 'abierta', 'independiente',
       'abierta, amueblada', 'independiente, amueblada',
       'independiente, equipada', 'equipada', 'amueblada',
       'independiente, amueblada, equipada', 'amplia',
       'amueblada, equipada', 'abierta, amueblada, equipada',
       'abierta, con isla', 'sin categoria', 'reformada',
       'abierta, equipada', 'amplia, equipada', 'amplia, amueblada',
       'no amueblada', 'Sin reforma', 'amplia, amueblada, equipada',
       'con isla', 'con isla, amueblada'], dtype=object)

In [2320]:
df_final.columns.to_list()

['title',
 'url',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'consumoce_ano',
 'letra_ce',
 'emisiones_co2',
 'tipologia',
 'estado',
 'amueblado',
 'planta',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'calefaccion',
 'antiguedad',
 'num_plantas',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'nuda_propiedad',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 'chimenea',
 'cocina',
 'fibra_optica',
 'exterior',
 'interior',
 'interior_y_exterior',
 'piscina',
 'porton_electrico',


In [2321]:
# Modificar los valores según la condición
df_final.loc[(df_final['tiene_armario'] == 1) & (df_final['armarios'] == 0), 'armarios'] = 1

In [2322]:

#eliminamos esta url , es un edificio

# Eliminar la fila donde la columna 'url' contiene la URL especificada
df_final = df_final[df_final['url'] != 'https://www.indomio.es/anuncios/93990143/']
df_final.loc[df_final['url'].str.contains('https://www.indomio.es/anuncios/93455821/'), ['tipologia', 'terraza', 'estado','garaje']] = ['Loft',1,'Buen estado',1]
df_final.loc[df_final["url"] == "https://www.indomio.es/anuncios/96654135/", "inmueble_ingresos"] = 1

urlsnuevo=['https://www.pisos.com/comprar/apartamento-adelfas28007-45835099732_101800/','https://www.pisos.com/comprar/apartamento-adelfas28007-45869202772_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-45912206979_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-45919489438_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-46667942689_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-46731956156_101800/']
#les asignamos una planta 5, los aticos son mas caros 
# Actualizar las filas correspondientes usando isin
df_final.loc[df_final['url'].isin(urlsnuevo), ['estado', 'trastero', 'tiene_piscina', 'garaje', 'planta']] = ['Obra nueva', 1, 1, 1, 5]

# se elimina la url https://www.pisos.com/comprar/piso-fontarron-47524209698_100600/ por que es un restaurante en venta
url_a_eliminar = 'https://www.pisos.com/comprar/piso-fontarron-47524209698_100600/'
df_final = df_final[df_final['url'] != url_a_eliminar]
#elimino esta fila , tiene datos que no ayudaran al modelo
df_final = df_final[df_final['url'] != 'https://www.indomio.es/anuncios/81008864/']

urls=['https://www.indomio.es/anuncios/96343641/','https://www.indomio.es/anuncios/96535879/']
df_final.loc[df_final['url'].isin(urls), 'num_plantas'] = 3

urls=['https://www.redpiso.es/inmueble/piso-en-venta-en-calle-alvarado-cuatro-caminos-tetuan-madrid-madrid-RP1362024130128',
      'https://www.redpiso.es/inmueble/estudio-en-venta-en-calle-bustamante-delicias-arganzuela-madrid-madrid-RP2102024125647']
df_final.loc[df_final['url'].isin(urls), 'planta'] = 3
# Asignar valores específicos a las columnas 'tipo_inmueble', 'estado' y 'num_garajes' para la URL indicada
df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/piso-goya28006-39193055272_102100/', ['tipo_inmueble', 'estado', 'garaje']] = ['lujo', 'Obra nueva', '4 privado']
df_final.loc[df_final['url'] == 'https://www.indomio.es/anuncios/95552665/', ['exterior', 'estado', 'garaje']] = [1, 'reformado', '4 privado']
df_final.loc[df_final['url'] == 'https://www.indomio.es/anuncios/96669269/', ['exterior', 'estado', 'grupo_cocina']] = [1, 'reformado', 'abierta, amueblada, equipada']
df_final.loc[df_final['url'] == 'https://www.indomio.es/anuncios/95491161/', ['exterior', 'estado', 'amueblado','grupo_cocina','aire_acondicionado','calefaccion']] = [1, 'reformado', '1','abierta',1,'central']
df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/duplex-recoletos28001-32597181586_996542/', ['exterior', 'estado', 'antiguedad']] = [1, 'reformado','Más de 50 años']
df_final.loc[df_final['url'] == 'https://www.redpiso.es/inmueble/piso-en-venta-en-paseo-castellana-almagro-chamberi-madrid-madrid-RP1922023117294', ['grupo_cocina','portero', 'estado', 'antiguedad','gastos_comunidad','amueblado','terraza','calefaccion']] = ['equipada',1, 'reformado',1954,1000,'1',1,'central']
df_final.loc[df_final['url'] == 'https://www.indomio.es/anuncios/96662699/', ['cod_barrio','barrio','cod_distrito','distrito','codigo_Postal','grupo_cocina','estado']] = [35,'Jerónimos',3,'Retiro',28014,'equipada','reformado']
df_final.loc[df_final['url'] == 'https://www.indomio.es/anuncios/82878580/', ["garaje",'chimenea',"grupo_cocina"]] = ["1 privado",1,"equipada independiente"]

df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/96736113/',["piscina","orientacion_norte","orientacion_sur","orientacion_este","orientacion_oeste","tipologia","portero","calefaccion","tipo_inmueble"]]=[1,1,1,1,1,"Atico",1,"central","lujo"]
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/95463509/',["estado"]]=['reformado']

df_final.loc[df_final['url']=='https://www.pisos.com/comprar/piso-recoletos28009-32538606854_149200/',["planta","estado","tipologia","exterior"]]=['bajo','reformado','Chalet',1]
df_final.loc[df_final['url']=='https://www.pisos.com/comprar/chalet-fuencarral_el_pardo_el_pardo28023-46668989227_102100/',["piscina","calefaccion","estado","tipologia","exterior","chimenea","jardin","tiene_jardin","cancha_tenis"]]=['propia','suelo radiante','perfecto estado','Chalet',1,1,"privado",'1',1]
df_final.loc[df_final['url']=='https://www.pisosmadrid.com.es/propiedad/4044649-estudio-en-venta-en-nueva-espana-madrid',"banos"]=1
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/93990153/',["garaje","trastero"]]=["2 privado",1]
df_final.loc[df_final['url']=='https://www.pisos.com/comprar/piso-casco_historico_de_vallecas28031-39169651748_101800/',["estado","calefaccion","puerta_seguridad","aire_acondicionado","grupo_cocina"]]=["buen estado","individual",1,"1","independiente equipada"]

df_final.loc[df_final['url']=='https://www.redpiso.es/inmueble/piso-en-venta-en-casco-historico-de-vallecas-villa-de-vallecas-ensanche-y-santa-eugenia-madrid-madrid-RP532024131243',["antiguedad","estado","letra_ce","terraza","trastero"]]=[1979,"reformado","E","1",0]
# Cambiar este valor por que no puede ser mayor los m2_utiles que los construidos
# Asignar NaN a 'm2_utiles' cuando 'url' es igual al valor indicado
df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/piso-centro_palacio28013-48428210263_100500/', 'm2_utiles'] = np.nan
# Cambiar este valor por que no puede ser mayor los m2_utiles que los construidos

df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/chalet-piovera28043-45869946604_108700/', 'm2_utiles'] = 1150
df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/piso-salamanca_castellana28006-41697553178_996152/', 'm2_utiles'] = 462
df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/piso-recoletos28001-44168048071_996152/', 'm2_utiles'] = 323



C:\Users\extas\AppData\Local\Temp\ipykernel_7920\2224392916.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4 privado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/piso-goya28006-39193055272_102100/', ['tipo_inmueble', 'estado', 'garaje']] = ['lujo', 'Obra nueva', '4 privado']
C:\Users\extas\AppData\Local\Temp\ipykernel_7920\2224392916.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[df_final['url']=='https://www.pisos.com/comprar/chalet-fuencarral_el_pardo_el_pardo28023-46668989227_102100/',["piscina","calefaccion","estado","tipologia","exterior","chimenea","jardin","tiene_jardin","cancha_tenis"]]=['p

In [2323]:
df_final["instalacion_calefaccion"].value_counts()

instalacion_calefaccion
Radiadores                 9725
No clasif                   268
Suelo Radiante               21
Aire                         14
No clasificado en medio       4
Name: count, dtype: int64

In [2324]:
df_final["tipo_suelo"].value_counts()

tipo_suelo
Parquet                              287
Tarima flotante                      253
 Gres                                147
 Parquet                             135
Tarima maciza                        113
Gres                                  97
 Tarima normal                        73
 Tarima Flotante                      62
Mármol                                28
 Baldosa                              21
 Sintasol                             16
 Terrazo                              16
 Otros                                14
 Porcelánico                           7
Terrazo                                7
Parket                                 6
Tarima                                 5
Madera                                 4
Moqueta                                4
Tarima normal                          3
 Mosaico Hidráulico                    3
Porcelanico                            2
Radiante-refrescante                   2
Cemento pulido                         1
Pino 

In [2325]:
# Asignar valores según las siguientes condiciones:
df_final['certificado_energetico'] = df_final['certificado_energetico'].replace({
    'Disponible': 1,
    'No indicado': 0
})
df_final['certificado_energetico'] = df_final['certificado_energetico'].fillna(0)

# Asegurarse de que todos los valores sean tratados de forma uniforme
df_final['certificado_energetico'] = df_final['certificado_energetico'].astype(float)

# También se puede reemplazar 0.0 por 0 (si quieres mantenerlo como entero)
df_final['certificado_energetico'] = df_final['certificado_energetico'].replace(0.0, 0)

# Verificar los cambios
print(df_final['certificado_energetico'].value_counts())

certificado_energetico
0.00    8144
1.00    1888
Name: count, dtype: int64


In [2326]:
# Supongamos que tienes un DataFrame llamado df
# Establecer el valor en 'tipo_suelo' como 'suelo radiante' si 'instalacion calefaccion' es 'suelo radiante'
df_final.loc[df_final['instalacion_calefaccion'] == 'Suelo Radiante', 'tipo_suelo'] = 'Suelo radiante'
df_final.loc[df_final['tipo_suelo'] == 'Suelo radiante', 'tipo_suelo'] = 'Suelo Radiante'

In [2327]:
df_final=df_final.drop(columns='num_garajes')
df_final=df_final.drop(columns='interior_y_exterior')

In [2328]:
cols=['amueblado','puerta_seguridad','piscina','jardin','garaje']
for c in cols:
  df_final[c]=df_final[c].astype(str)

In [2329]:

cols = ['url', 'descripcion_larga']
dfCl_ChatGPT = df_final[cols]



# Ruta relativa del archivo
ruta_guardado_relativa = '../../03-clustering/data/pisosCl_CharGPT.csv'

# Crear el directorio de destino si no existe
directorio_destino = os.path.dirname(ruta_guardado_relativa)
os.makedirs(directorio_destino, exist_ok=True)

# Guardar el DataFrame dfEDA en la ruta relativa especificada
dfCl_ChatGPT.to_csv(ruta_guardado_relativa, index=False)

In [2330]:
columnas_eliminar=["web",'tiene_piscina',"tiene_jardin",'title','precio_anterior','EUR/m2','descuento','zona','calle','emisiones_co2','referencia',"fecha_publicacion","titular_anuncio",'titularanuncio',"cocina",
"nombre_oficina","num_visitas" ,'alarma','num_pisos','descuento','consumoce_ano','emisiones_ano', 'calefaccion','aire_acondicionado','fecha_descarga','actualizacion','descripcion_larga','zonas_verdes','nombre_oficina','fibra_optica']

df_final.reset_index(drop=True, inplace=True)


dfEDA = df_final.drop(columns=columnas_eliminar)

# Ruta relativa del archivo
ruta_guardado_relativa = '../../02-EDA-y-feature-engineering/data/pisosEDA.csv'

# Crear el directorio de destino si no existe
directorio_destino = os.path.dirname(ruta_guardado_relativa)
os.makedirs(directorio_destino, exist_ok=True)

# Guardar el DataFrame dfEDA en la ruta relativa especificada
dfEDA.to_csv(ruta_guardado_relativa,sep=";", index=False)

In [2331]:
# Especificar las columnas para verificar los valores únicos
columns_to_check = [9,  15, 20, 38, 48]

# Obtener los valores únicos para cada columna especificada
unique_values = {col: dfEDA.iloc[:, col].unique() for col in columns_to_check}

# Imprimir los valores únicos
for col, values in unique_values.items():
    print(f"Columna {col}: {values}")

Columna 9: ['Sólo cocina amueblada' 'Sí' 'nan' 'Parcialmente amueblado' 'No' '1'
 '1.0']
Columna 15: ['1.0' 'nan' '1 privado' '2 privado' '0.0' '4 privado']
Columna 20: ['nan' 'sin jardin' 'privado' 'comunitario']
Columna 38: ['1' 'sin piscina' 'comunitaria' 'None' 'propia' 'privada'
 '1 piscina de adultos y 1 piscina de ninos' 'con salorium' '1.0']
Columna 48: ['1' '0' '<NA>']


#revisar este codigo

In [2332]:
dfEDA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10032 entries, 0 to 10031
Data columns (total 63 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   url                                        10032 non-null  object 
 1   m2_constr                                  10032 non-null  float64
 2   distrito                                   10032 non-null  object 
 3   cod_distrito                               10032 non-null  Int64  
 4   barrio                                     10032 non-null  object 
 5   cod_barrio                                 10032 non-null  Int64  
 6   letra_ce                                   2150 non-null   object 
 7   tipologia                                  10032 non-null  object 
 8   estado                                     5525 non-null   object 
 9   amueblado                                  10032 non-null  object 
 10  planta                

In [2333]:
dfEDA.shape

(10032, 63)

In [2334]:
dfEDA.columns.tolist()

['url',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'letra_ce',
 'tipologia',
 'estado',
 'amueblado',
 'planta',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'antiguedad',
 'num_plantas',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'nuda_propiedad',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 'chimenea',
 'exterior',
 'interior',
 'piscina',
 'porton_electrico',
 'videoportero',
 'acceso_discapacidad',
 'precio',
 'armarios',
 'codigo_Postal',
 'm2_utiles',
 'tipo_suelo',
 'certificado_energetico',
 'puerta_seguridad',
 'gastos_comunidad',
 'tipo_fachada',
 'a

In [2335]:
# Forzar la recolección de basura
gc.collect()


14